# MOUNT DRIVE

In [32]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

# IMPORT LIBRERIE

In [33]:
pip install timm

Error processing line 1 of /home/ale-ubuntu/anaconda3/lib/python3.7/site-packages/vision-1.0.0-py3.7-nspkg.pth:

  Traceback (most recent call last):
    File "/home/ale-ubuntu/anaconda3/lib/python3.7/site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 580, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install einops

Error processing line 1 of /home/ale-ubuntu/anaconda3/lib/python3.7/site-packages/vision-1.0.0-py3.7-nspkg.pth:

  Traceback (most recent call last):
    File "/home/ale-ubuntu/anaconda3/lib/python3.7/site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 580, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
Note: you may need to restart the kernel to use updated packages.


# HELPERS

In [35]:
%%writefile helpers.py

import math
import torch
import torch.nn.functional as F
import logging

_logger = logging.getLogger('train')


def resize_pos_embed(posemb, posemb_new, num_tokens=1):
    # Copied from `timm` by Ross Wightman:
    # github.com/rwightman/pytorch-image-models
    # Rescale the grid of position embeddings when loading from state_dict. Adapted from
    # https://github.com/google-research/vision_transformer/blob/00883dd691c63a6830751563748663526e811cee/vit_jax/checkpoint.py#L224
    ntok_new = posemb_new.shape[1]
    if num_tokens:
        posemb_tok, posemb_grid = posemb[:, :num_tokens], posemb[0, num_tokens:]
        ntok_new -= num_tokens
    else:
        posemb_tok, posemb_grid = posemb[:, :0], posemb[0]
    gs_old = int(math.sqrt(len(posemb_grid)))
    gs_new = int(math.sqrt(ntok_new))
    posemb_grid = posemb_grid.reshape(1, gs_old, gs_old, -1).permute(0, 3, 1, 2)
    posemb_grid = F.interpolate(posemb_grid, size=(gs_new, gs_new), mode='bilinear')
    posemb_grid = posemb_grid.permute(0, 2, 3, 1).reshape(1, gs_new * gs_new, -1)
    posemb = torch.cat([posemb_tok, posemb_grid], dim=1)
    return posemb


def pe_check(model, state_dict, pe_key='classifier.positional_emb'):
    if pe_key is not None and pe_key in state_dict.keys() and pe_key in model.state_dict().keys():
        if model.state_dict()[pe_key].shape != state_dict[pe_key].shape:
            state_dict[pe_key] = resize_pos_embed(state_dict[pe_key],
                                                  model.state_dict()[pe_key],
                                                  num_tokens=model.classifier.num_tokens)
    return state_dict


def fc_check(model, state_dict, fc_key='classifier.fc'):
    for key in [f'{fc_key}.weight', f'{fc_key}.bias']:
        if key is not None and key in state_dict.keys() and key in model.state_dict().keys():
            if model.state_dict()[key].shape != state_dict[key].shape:
                _logger.warning(f'Removing {key}, number of classes has changed.')
                state_dict[key] = model.state_dict()[key]
    return state_dict


Overwriting helpers.py


In [36]:
def register_model(func):
    """
    Fallback wrapper in case timm isn't installed
    """
    return func

In [51]:
%%writefile checkpoint.py

import glob
import operator
import os
import logging

import torch

from timm.utils.model import unwrap_model, get_state_dict


_logger = logging.getLogger(__name__)


class CheckpointSaver:
    def __init__(
            self,
            model,
            optimizer,
            args=None,
            model_ema=None,
            amp_scaler=None,
            checkpoint_prefix='checkpoint',
            recovery_prefix='recovery',
            checkpoint_dir='',
            recovery_dir='',
            decreasing=False,
            max_history=10,
            unwrap_fn=unwrap_model):

        # objects to save state_dicts of
        self.model = model
        self.optimizer = optimizer
        self.args = args
        self.model_ema = model_ema
        self.amp_scaler = amp_scaler

        # state
        self.checkpoint_files = []  # (filename, metric) tuples in order of decreasing betterness
        self.best_epoch = None
        self.best_metric = None
        self.curr_recovery_file = ''
        self.last_recovery_file = ''

        # config
        self.checkpoint_dir = checkpoint_dir
        self.recovery_dir = recovery_dir
        self.save_prefix = checkpoint_prefix
        self.recovery_prefix = recovery_prefix
        self.extension = '.pth.tar'
        self.decreasing = decreasing  # a lower metric is better if True
        self.cmp = operator.lt if decreasing else operator.gt  # True if lhs better than rhs
        self.max_history = max_history
        self.unwrap_fn = unwrap_fn
        assert self.max_history >= 1

    def save_checkpoint(self, epoch, metric=None):
        assert epoch >= 0
        last_save_path = os.path.join(self.checkpoint_dir, 'last' + self.extension)
        #self._save(last_save_path, epoch, metric)
        worst_file = self.checkpoint_files[-1] if self.checkpoint_files else None
        if (len(self.checkpoint_files) < self.max_history
                or metric is None or self.cmp(metric, worst_file[1])):
            if len(self.checkpoint_files) >= self.max_history:
                self._cleanup_checkpoints(1)
            filename = '-'.join([self.save_prefix, str(epoch)]) + self.extension
            save_path = os.path.join(self.checkpoint_dir, filename)
            #self._save(save_path, epoch, metric)
            self.checkpoint_files.append((save_path, metric))
            self.checkpoint_files = sorted(
                self.checkpoint_files, key=lambda x: x[1],
                reverse=not self.decreasing)  # sort in descending order if a lower metric is not better

            if metric is not None and (self.best_metric is None or self.cmp(metric, self.best_metric)):
              self.best_epoch = epoch
              self.best_metric = metric
              best_save_path = os.path.join(self.checkpoint_dir, 'model_best' + self.extension)
              self._save(best_save_path, epoch, metric)

            checkpoints_str = "Current checkpoints:\n"
            for c in self.checkpoint_files:
                checkpoints_str += ' {}\n'.format(c)
            _logger.info(checkpoints_str)

        return (None, None) if self.best_metric is None else (self.best_metric, self.best_epoch)

    def _save(self, save_path, epoch, metric=None):
        save_state = {
            'epoch': epoch,
            'arch': type(self.model).__name__.lower(),
            'state_dict': get_state_dict(self.model, self.unwrap_fn),
            'optimizer': self.optimizer.state_dict(),
            'version': 2,  # version < 2 increments epoch before save
        }
        if self.args is not None:
            save_state['arch'] = self.args.model
            save_state['args'] = self.args
        if self.amp_scaler is not None:
            save_state[self.amp_scaler.state_dict_key] = self.amp_scaler.state_dict()
        if self.model_ema is not None:
            save_state['state_dict_ema'] = get_state_dict(self.model_ema, self.unwrap_fn)
        if metric is not None:
            save_state['metric'] = metric
        torch.save(save_state, save_path)

    def _cleanup_checkpoints(self, trim=0):
        trim = min(len(self.checkpoint_files), trim)
        delete_index = self.max_history - trim
        if delete_index < 0 or len(self.checkpoint_files) <= delete_index:
            return
        to_delete = self.checkpoint_files[delete_index:]
        for d in to_delete:
            try:
                _logger.debug("Cleaning checkpoint: {}".format(d))
                os.remove(d[0])
            except Exception as e:
                _logger.error("Exception '{}' while deleting checkpoint".format(e))
        self.checkpoint_files = self.checkpoint_files[:delete_index]

    def save_recovery(self, epoch, batch_idx=0):
        assert epoch >= 0
        filename = '-'.join([self.recovery_prefix, str(epoch), str(batch_idx)]) + self.extension
        save_path = os.path.join(self.recovery_dir, filename)
        self._save(save_path, epoch)
        if os.path.exists(self.last_recovery_file):
            try:
                _logger.debug("Cleaning recovery: {}".format(self.last_recovery_file))
                os.remove(self.last_recovery_file)
            except Exception as e:
                _logger.error("Exception '{}' while removing {}".format(e, self.last_recovery_file))
        self.last_recovery_file = self.curr_recovery_file
        self.curr_recovery_file = save_path

    def find_recovery(self):
        recovery_path = os.path.join(self.recovery_dir, self.recovery_prefix)
        files = glob.glob(recovery_path + '*' + self.extension)
        files = sorted(files)
        return files[0] if len(files) else ''


Overwriting checkpoint.py


# TOKENIZERS

Tokenizer



In [38]:
%%writefile tokenizer.py

import torch
import torch.nn as nn
import torch.nn.functional as F


class Tokenizer(nn.Module):
    def __init__(self,
                 kernel_size, stride, padding,
                 pooling_kernel_size=3, pooling_stride=2, pooling_padding=1,
                 n_conv_layers=1,
                 n_input_channels=3,
                 n_output_channels=64,
                 in_planes=64,
                 activation=None,
                 max_pool=True,
                 conv_bias=False):
        super(Tokenizer, self).__init__()

        n_filter_list = [n_input_channels] + \
                        [in_planes for _ in range(n_conv_layers - 1)] + \
                        [n_output_channels]

        self.conv_layers = nn.Sequential(
            *[nn.Sequential(
                nn.Conv2d(n_filter_list[i], n_filter_list[i + 1],
                          kernel_size=(kernel_size, kernel_size),
                          stride=(stride, stride),
                          padding=(padding, padding), bias=conv_bias),
                nn.Identity() if activation is None else activation(),
                nn.MaxPool2d(kernel_size=pooling_kernel_size,
                             stride=pooling_stride,
                             padding=pooling_padding) if max_pool else nn.Identity()
            )
                for i in range(n_conv_layers)
            ])

        self.flattener = nn.Flatten(2, 3)
        self.apply(self.init_weight)

    def sequence_length(self, n_channels=3, height=224, width=224):
        return self.forward(torch.zeros((1, n_channels, height, width))).shape[1]

    def forward(self, x):
        return self.flattener(self.conv_layers(x)).transpose(-2, -1)

    @staticmethod
    def init_weight(m):
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight)


class TextTokenizer(nn.Module):
    def __init__(self,
                 kernel_size, stride, padding,
                 pooling_kernel_size=3, pooling_stride=2, pooling_padding=1,
                 embedding_dim=300,
                 n_output_channels=128,
                 activation=None,
                 max_pool=True,
                 *args, **kwargs):
        super(TextTokenizer, self).__init__()

        self.max_pool = max_pool
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, n_output_channels,
                      kernel_size=(kernel_size, embedding_dim),
                      stride=(stride, 1),
                      padding=(padding, 0), bias=False),
            nn.Identity() if activation is None else activation(),
            nn.MaxPool2d(
                kernel_size=(pooling_kernel_size, 1),
                stride=(pooling_stride, 1),
                padding=(pooling_padding, 0)
            ) if max_pool else nn.Identity()
        )

        self.apply(self.init_weight)

    def seq_len(self, seq_len=32, embed_dim=300):
        return self.forward(torch.zeros((1, seq_len, embed_dim)))[0].shape[1]

    def forward_mask(self, mask):
        new_mask = mask.unsqueeze(1).float()
        cnn_weight = torch.ones(
            (1, 1, self.conv_layers[0].kernel_size[0]),
            device=mask.device,
            dtype=torch.float)
        new_mask = F.conv1d(
            new_mask, cnn_weight, None,
            self.conv_layers[0].stride[0], self.conv_layers[0].padding[0], 1, 1)
        if self.max_pool:
            new_mask = F.max_pool1d(
                new_mask, self.conv_layers[2].kernel_size[0],
                self.conv_layers[2].stride[0], self.conv_layers[2].padding[0], 1, False, False)
        new_mask = new_mask.squeeze(1)
        new_mask = (new_mask > 0)
        return new_mask

    def forward(self, x, mask=None):
        x = x.unsqueeze(1)
        x = self.conv_layers(x)
        x = x.transpose(1, 3).squeeze(1)
        if mask is not None:
            mask = self.forward_mask(mask).unsqueeze(-1).float()
            x = x * mask
        return x, mask

    @staticmethod
    def init_weight(m):
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight)


Overwriting tokenizer.py


Memory Tokenizer

In [39]:
import torch.nn as nn
from einops.layers.torch import Rearrange
import torch

class VerticalTokenizer(nn.Module):
    """
    This class contains the module of the ViT that generates vertical patch embeddings from the input image.
    Attributes:
    projection        object of nn.Sequential class that splits the image into patches
                      and generates projections in the size of the embedding
    positions         object of nn.Parameter class that generates the positional encoding vectors to add to all patch
                      embeddings
    """
    def __init__(self, img_size, width_patches, dim, channels=3):
        super().__init__()
        self.projection = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=width_patches, p2=img_size),
            nn.Linear(channels * img_size * width_patches, dim),
        )

    def forward(self, x):
        x = self.projection(x)
        return x

    def sequence_length(self, n_channels=3, height=224, width=224):
        return self.forward(torch.zeros((1, n_channels, height, width))).shape[1]

    @staticmethod
    def init_weight(m):
        if isinstance(m, nn.Sequential):
            nn.init.kaiming_normal_(m.weight)

# TRANSFORMERS

In [40]:
%%writefile transformers_component.py

# Thanks to rwightman's timm package
# github.com:rwightman/pytorch-image-models

import torch
import torch.nn as nn
from torch.nn import Module, ModuleList, Linear, Dropout, LayerNorm, Identity, Parameter, init
import torch.nn.functional as F


def drop_path(x, drop_prob: float = 0., training: bool = False):
    """
    Obtained from: github.com:rwightman/pytorch-image-models
    Drop paths (Stochastic Depth) per sample (when applied in main path of residual blocks).
    This is the same as the DropConnect impl I created for EfficientNet, etc networks, however,
    the original name is misleading as 'Drop Connect' is a different form of dropout in a separate paper...
    See discussion: https://github.com/tensorflow/tpu/issues/494#issuecomment-532968956 ... I've opted for
    changing the layer and argument names to 'drop path' rather than mix DropConnect as a layer name and use
    'survival rate' as the argument.
    """
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # work with diff dim tensors, not just 2D ConvNets
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    random_tensor.floor_()  # binarize
    output = x.div(keep_prob) * random_tensor
    return output


class DropPath(nn.Module):
    """
    Obtained from: github.com:rwightman/pytorch-image-models
    Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).
    """

    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)



class Attention(Module):
    """
    Obtained from timm: github.com:rwightman/pytorch-image-models
    """

    def __init__(self, dim, num_heads=8, attention_dropout=0.1, projection_dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // self.num_heads
        self.scale = head_dim ** -0.5

        self.qkv = Linear(dim, dim * 3, bias=False)
        self.attn_drop = Dropout(attention_dropout)
        self.proj = Linear(dim, dim)
        self.proj_drop = Dropout(projection_dropout)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class MaskedAttention(Module):
    def __init__(self, dim, num_heads=8, attention_dropout=0.1, projection_dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // self.num_heads
        self.scale = head_dim ** -0.5

        self.qkv = Linear(dim, dim * 3, bias=False)
        self.attn_drop = Dropout(attention_dropout)
        self.proj = Linear(dim, dim)
        self.proj_drop = Dropout(projection_dropout)

    def forward(self, x, mask=None):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q @ k.transpose(-2, -1)) * self.scale

        if mask is not None:
            mask_value = -torch.finfo(attn.dtype).max
            assert mask.shape[-1] == attn.shape[-1], 'mask has incorrect dimensions'
            mask = mask[:, None, :] * mask[:, :, None]
            mask = mask.unsqueeze(1).repeat(1, self.num_heads, 1, 1)
            attn.masked_fill_(~mask, mask_value)

        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class TransformerEncoderLayer(Module):
    """
    Inspired by torch.nn.TransformerEncoderLayer and timm.
    """

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1,
                 attention_dropout=0.1, drop_path_rate=0.1):
        super(TransformerEncoderLayer, self).__init__()
        self.pre_norm = LayerNorm(d_model)
        self.self_attn = Attention(dim=d_model, num_heads=nhead,
                                   attention_dropout=attention_dropout, projection_dropout=dropout)

        self.linear1 = Linear(d_model, dim_feedforward)
        self.dropout1 = Dropout(dropout)
        self.norm1 = LayerNorm(d_model)
        self.linear2 = Linear(dim_feedforward, d_model)
        self.dropout2 = Dropout(dropout)

        self.drop_path = DropPath(drop_path_rate) if drop_path_rate > 0 else Identity()

        self.activation = F.gelu

    def forward(self, src: torch.Tensor, *args, **kwargs) -> torch.Tensor:
        src = src + self.drop_path(self.self_attn(self.pre_norm(src)))
        src = self.norm1(src)
        src2 = self.linear2(self.dropout1(self.activation(self.linear1(src))))
        src = src + self.drop_path(self.dropout2(src2))
        return src


class MaskedTransformerEncoderLayer(Module):
    """
    Inspired by torch.nn.TransformerEncoderLayer and timm.
    """

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1,
                 attention_dropout=0.1, drop_path_rate=0.1):
        super(MaskedTransformerEncoderLayer, self).__init__()
        self.pre_norm = LayerNorm(d_model)
        self.self_attn = MaskedAttention(dim=d_model, num_heads=nhead,
                                         attention_dropout=attention_dropout, projection_dropout=dropout)

        self.linear1 = Linear(d_model, dim_feedforward)
        self.dropout1 = Dropout(dropout)
        self.norm1 = LayerNorm(d_model)
        self.linear2 = Linear(dim_feedforward, d_model)
        self.dropout2 = Dropout(dropout)

        self.drop_path = DropPath(drop_path_rate) if drop_path_rate > 0 else Identity()

        self.activation = F.gelu

    def forward(self, src: torch.Tensor, mask=None, *args, **kwargs) -> torch.Tensor:
        src = src + self.drop_path(self.self_attn(self.pre_norm(src), mask))
        src = self.norm1(src)
        src2 = self.linear2(self.dropout1(self.activation(self.linear1(src))))
        src = src + self.drop_path(self.dropout2(src2))
        return src


class TransformerClassifier(Module):
    def __init__(self,
                 seq_pool=True,
                 embedding_dim=768,
                 num_layers=12,
                 num_heads=12,
                 mlp_ratio=4.0,
                 num_classes=1000,
                 dropout=0.1,
                 attention_dropout=0.1,
                 stochastic_depth=0.1,
                 positional_embedding='learnable',
                 sequence_length=None):
        super().__init__()
        positional_embedding = positional_embedding if \
            positional_embedding in ['sine', 'learnable', 'none'] else 'sine'
        dim_feedforward = int(embedding_dim * mlp_ratio)
        self.embedding_dim = embedding_dim
        self.sequence_length = sequence_length
        self.seq_pool = seq_pool
        self.num_tokens = 0

        assert sequence_length is not None or positional_embedding == 'none', \
            f"Positional embedding is set to {positional_embedding} and" \
            f" the sequence length was not specified."

        if not seq_pool:
            sequence_length += 1
            self.class_emb = Parameter(torch.zeros(1, 1, self.embedding_dim),
                                       requires_grad=True)
            self.num_tokens = 1
        else:
            self.attention_pool = Linear(self.embedding_dim, 1)

        if positional_embedding != 'none':
            if positional_embedding == 'learnable':
                self.positional_emb = Parameter(torch.zeros(1, sequence_length, embedding_dim),
                                                requires_grad=True)
                init.normal_(self.positional_emb, std=0.2)
            else:
                self.positional_emb = Parameter(self.sinusoidal_embedding(sequence_length, embedding_dim),
                                                requires_grad=False)
        else:
            self.positional_emb = None

        self.dropout = Dropout(p=dropout)
        dpr = [x.item() for x in torch.linspace(0, stochastic_depth, num_layers)]
        self.blocks = ModuleList([
            TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads,
                                    dim_feedforward=dim_feedforward, dropout=dropout,
                                    attention_dropout=attention_dropout, drop_path_rate=dpr[i])
            for i in range(num_layers)])
        self.norm = LayerNorm(embedding_dim)

        self.fc = Linear(embedding_dim, num_classes)
        self.apply(self.init_weight)

    def forward(self, x):
        if self.positional_emb is None and x.size(1) < self.sequence_length:
            x = F.pad(x, (0, 0, 0, self.n_channels - x.size(1)), mode='constant', value=0)

        if not self.seq_pool:
            cls_token = self.class_emb.expand(x.shape[0], -1, -1)
            x = torch.cat((cls_token, x), dim=1)

        if self.positional_emb is not None:
            x += self.positional_emb

        x = self.dropout(x)

        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)

        if self.seq_pool:
            x = torch.matmul(F.softmax(self.attention_pool(x), dim=1).transpose(-1, -2), x).squeeze(-2)
        else:
            x = x[:, 0]

        x = self.fc(x)
        return x

    @staticmethod
    def init_weight(m):
        if isinstance(m, Linear):
            init.normal_(m.weight, std=.02)
            if isinstance(m, Linear) and m.bias is not None:
                init.constant_(m.bias, 0)
        elif isinstance(m, LayerNorm):
            init.constant_(m.bias, 0)
            init.constant_(m.weight, 1.0)

    @staticmethod
    def sinusoidal_embedding(n_channels, dim):
        pe = torch.FloatTensor([[p / (10000 ** (2 * (i // 2) / dim)) for i in range(dim)]
                                for p in range(n_channels)])
        pe[:, 0::2] = torch.sin(pe[:, 0::2])
        pe[:, 1::2] = torch.cos(pe[:, 1::2])
        return pe.unsqueeze(0)


class MaskedTransformerClassifier(Module):
    def __init__(self,
                 seq_pool=True,
                 embedding_dim=768,
                 num_layers=12,
                 num_heads=12,
                 mlp_ratio=4.0,
                 num_classes=1000,
                 dropout=0.1,
                 attention_dropout=0.1,
                 stochastic_depth=0.1,
                 positional_embedding='sine',
                 seq_len=None,
                 *args, **kwargs):
        super().__init__()
        positional_embedding = positional_embedding if \
            positional_embedding in ['sine', 'learnable', 'none'] else 'sine'
        dim_feedforward = int(embedding_dim * mlp_ratio)
        self.embedding_dim = embedding_dim
        self.seq_len = seq_len
        self.seq_pool = seq_pool
        self.num_tokens = 0

        assert seq_len is not None or positional_embedding == 'none', \
            f"Positional embedding is set to {positional_embedding} and" \
            f" the sequence length was not specified."

        if not seq_pool:
            seq_len += 1
            self.class_emb = Parameter(torch.zeros(1, 1, self.embedding_dim),
                                       requires_grad=True)
            self.num_tokens = 1
        else:
            self.attention_pool = Linear(self.embedding_dim, 1)

        if positional_embedding != 'none':
            if positional_embedding == 'learnable':
                seq_len += 1  # padding idx
                self.positional_emb = Parameter(torch.zeros(1, seq_len, embedding_dim),
                                                requires_grad=True)
                init.trunc_normal_(self.positional_emb, std=0.2)
            else:
                self.positional_emb = Parameter(self.sinusoidal_embedding(seq_len,
                                                                          embedding_dim,
                                                                          padding_idx=True),
                                                requires_grad=False)
        else:
            self.positional_emb = None

        self.dropout = Dropout(p=dropout)
        dpr = [x.item() for x in torch.linspace(0, stochastic_depth, num_layers)]
        self.blocks = ModuleList([
            MaskedTransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads,
                                          dim_feedforward=dim_feedforward, dropout=dropout,
                                          attention_dropout=attention_dropout, drop_path_rate=dpr[i])
            for i in range(num_layers)])
        self.norm = LayerNorm(embedding_dim)

        self.fc = Linear(embedding_dim, num_classes)
        self.apply(self.init_weight)

    def forward(self, x, mask=None):
        if self.positional_emb is None and x.size(1) < self.seq_len:
            x = F.pad(x, (0, 0, 0, self.n_channels - x.size(1)), mode='constant', value=0)

        if not self.seq_pool:
            cls_token = self.class_emb.expand(x.shape[0], -1, -1)
            x = torch.cat((cls_token, x), dim=1)
            if mask is not None:
                mask = torch.cat([torch.ones(size=(mask.shape[0], 1), device=mask.device), mask.float()], dim=1)
                mask = (mask > 0)

        if self.positional_emb is not None:
            x += self.positional_emb

        x = self.dropout(x)

        for blk in self.blocks:
            x = blk(x, mask=mask)
        x = self.norm(x)

        if self.seq_pool:
            x = torch.matmul(F.softmax(self.attention_pool(x), dim=1).transpose(-1, -2), x).squeeze(-2)
        else:
            x = x[:, 0]

        x = self.fc(x)
        return x

    @staticmethod
    def init_weight(m):
        if isinstance(m, Linear):
            init.trunc_normal_(m.weight, std=.02)
            if isinstance(m, Linear) and m.bias is not None:
                init.constant_(m.bias, 0)
        elif isinstance(m, LayerNorm):
            init.constant_(m.bias, 0)
            init.constant_(m.weight, 1.0)

    @staticmethod
    def sinusoidal_embedding(n_channels, dim, padding_idx=False):
        pe = torch.FloatTensor([[p / (10000 ** (2 * (i // 2) / dim)) for i in range(dim)]
                                for p in range(n_channels)])
        pe[:, 0::2] = torch.sin(pe[:, 0::2])
        pe[:, 1::2] = torch.cos(pe[:, 1::2])
        pe = pe.unsqueeze(0)
        if padding_idx:
            return torch.cat([torch.zeros((1, 1, dim)), pe], dim=1)
        return pe



Overwriting transformers_component.py


Transformer Memory

In [41]:
from torch.nn import Module
class TransformerMemoryClassifier(Module):
    def __init__(self,
                 embedding_dim=768,
                 num_layers=12,
                 num_heads=12,
                 mlp_ratio=4.0,
                 num_classes=1000,
                 dropout=0.1,
                 attention_dropout=0.1,
                 stochastic_depth=0.1,
                 positional_embedding='learnable',
                 sequence_length=None):
        super().__init__()
        positional_embedding = positional_embedding if \
            positional_embedding in ['sine', 'learnable', 'none'] else 'sine'
        dim_feedforward = int(embedding_dim * mlp_ratio)
        self.embedding_dim = embedding_dim
        self.sequence_length = sequence_length
        self.num_tokens = 0

        assert sequence_length is not None or positional_embedding == 'none', \
            f"Positional embedding is set to {positional_embedding} and" \
            f" the sequence length was not specified."

        self.attention_pool = Linear(self.embedding_dim, 1)

        if positional_embedding != 'none':
            if positional_embedding == 'learnable':
                self.positional_emb = Parameter(torch.zeros(1, sequence_length, embedding_dim),
                                                requires_grad=True)
                init.normal_(self.positional_emb, std=0.2)
            else:
                self.positional_emb = Parameter(self.sinusoidal_embedding(sequence_length, embedding_dim),
                                                requires_grad=False)
        else:
            self.positional_emb = None

        self.dropout = Dropout(p=dropout)
        dpr = [x.item() for x in torch.linspace(0, stochastic_depth, num_layers)]
        self.blocks = ModuleList([
            TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads,
                                    dim_feedforward=dim_feedforward, dropout=dropout,
                                    attention_dropout=attention_dropout, drop_path_rate=dpr[i])
            for i in range(num_layers)])
        self.lstm = LSTM(embedding_dim, embedding_dim)
        self.norm = LayerNorm(embedding_dim)
        self.fc = Linear(embedding_dim, num_classes)
        self.apply(self.init_weight)

    def forward(self, x):
        if self.positional_emb is None and x.size(1) < self.sequence_length:
            x = F.pad(x, (0, 0, 0, self.n_channels - x.size(1)), mode='constant', value=0)

        if self.positional_emb is not None:
            x += self.positional_emb

        x = self.dropout(x)

        for blk in self.blocks:
            x = blk(x)
        x = self.lstm(x)[0]
        x = self.norm(x)

        x = torch.matmul(F.softmax(self.attention_pool(x), dim=1).transpose(-1, -2), x).squeeze(-2)

        x = self.fc(x)
        return x

    @staticmethod
    def init_weight(m):
        if isinstance(m, Linear):
            init.normal_(m.weight, std=.02)
            if isinstance(m, Linear) and m.bias is not None:
                init.constant_(m.bias, 0)
        elif isinstance(m, LayerNorm):
            init.constant_(m.bias, 0)
            init.constant_(m.weight, 1.0)

    @staticmethod
    def sinusoidal_embedding(n_channels, dim):
        pe = torch.FloatTensor([[p / (10000 ** (2 * (i // 2) / dim)) for i in range(dim)]
                                for p in range(n_channels)])
        pe[:, 0::2] = torch.sin(pe[:, 0::2])
        pe[:, 1::2] = torch.cos(pe[:, 1::2])
        return pe.unsqueeze(0)

    def extract_embedding(self, x):
        if self.positional_emb is None and x.size(1) < self.sequence_length:
            x = F.pad(x, (0, 0, 0, self.n_channels - x.size(1)), mode='constant', value=0)

        if self.positional_emb is not None:
            x += self.positional_emb

        x = self.dropout(x)

        for blk in self.blocks:
            x = blk(x)
        x = self.lstm(x)[0]
        x = self.norm(x)

        x = torch.matmul(F.softmax(self.attention_pool(x), dim=1).transpose(-1, -2), x).squeeze(-2)

        return x

    def extract_memory_embedding(self, x):
        if self.positional_emb is None and x.size(1) < self.sequence_length:
            x = F.pad(x, (0, 0, 0, self.n_channels - x.size(1)), mode='constant', value=0)

        if self.positional_emb is not None:
            x += self.positional_emb

        x = self.dropout(x)

        for blk in self.blocks:
            x = blk(x)
        x = self.lstm(x)[0]
        x = self.norm(x)

        return x


Transformer Parallel Memory Classifier

In [42]:

class ParallelMemoryClassifier(Module):
    def __init__(self,
                 embedding_dim=768,
                 num_layers=12,
                 num_heads=12,
                 mlp_ratio=4.0,
                 num_classes=1000,
                 dropout=0.1,
                 attention_dropout=0.1,
                 stochastic_depth=0.1,
                 positional_embedding='learnable',
                 sequence_length=None,
                 type='concatenate'):
        super().__init__()
        positional_embedding = positional_embedding if \
            positional_embedding in ['sine', 'learnable', 'none'] else 'sine'
        dim_feedforward = int(embedding_dim * mlp_ratio)
        self.embedding_dim = embedding_dim
        self.sequence_length = sequence_length
        self.num_tokens = 0

        assert sequence_length is not None or positional_embedding == 'none', \
            f"Positional embedding is set to {positional_embedding} and" \
            f" the sequence length was not specified."

        assert type == 'concatenate' or type == 'sum' or type == 'average', \
            f"Union output type is set to {type} that is not specified"
        self.union_type = type

        self.attention_pool_cct = Linear(self.embedding_dim, 1)
        self.attention_pool_lstm = Linear(self.embedding_dim, 1)

        if positional_embedding != 'none':
            if positional_embedding == 'learnable':
                self.positional_emb = Parameter(torch.zeros(1, sequence_length, embedding_dim),
                                                requires_grad=True)
                init.normal_(self.positional_emb, std=0.2)
            else:
                self.positional_emb = Parameter(self.sinusoidal_embedding(sequence_length, embedding_dim),
                                                requires_grad=False)
        else:
            self.positional_emb = None

        self.dropout = Dropout(p=dropout)
        dpr = [x.item() for x in torch.linspace(0, stochastic_depth, num_layers)]
        self.blocks = ModuleList([
            TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads,
                                    dim_feedforward=dim_feedforward, dropout=dropout,
                                    attention_dropout=attention_dropout, drop_path_rate=dpr[i])
            for i in range(num_layers)])
        self.norm_cct = LayerNorm(embedding_dim)

        self.lstm = LSTM(embedding_dim, embedding_dim)
        self.norm_lstm = LayerNorm(embedding_dim)
        fc_dim = embedding_dim
        if self.union_type == 'concatenate':
            fc_dim = embedding_dim * 2

        self.norm_union = LayerNorm(fc_dim)
        self.fc = Linear(fc_dim, num_classes)
        self.apply(self.init_weight)

    def forward(self, x_cct, x_lstm):
        if self.positional_emb is None and x_cct.size(1) < self.sequence_length:
            x_cct = F.pad(x_cct, (0, 0, 0, self.n_channels - x_cct.size(1)), mode='constant', value=0)

        if self.positional_emb is not None:
            x_cct += self.positional_emb

        x_cct = self.dropout(x_cct)

        for blk in self.blocks:
            x_cct = blk(x_cct)

        x_cct = self.norm_cct(x_cct)
        x_cct = torch.matmul(F.softmax(self.attention_pool_cct(x_cct), dim=1).transpose(-1, -2), x_cct).squeeze(-2)

        x_lstm = self.lstm(x_lstm)[0]
        x_lstm = self.norm_lstm(x_lstm)

        x_lstm = torch.matmul(F.softmax(self.attention_pool_lstm(x_lstm), dim=1).transpose(-1, -2), x_lstm).squeeze(-2)
        if self.union_type == 'concatenate':
            x = torch.cat((x_cct, x_lstm), 1)
        elif self.union_type == 'sum':
            x = torch.sum(torch.stack([x_cct, x_lstm]), dim=0)
        elif self.union_type == 'average':
            x = torch.mean(torch.stack([x_cct, x_lstm]), dim=0)

        x = self.norm_union(x)

        x = self.fc(x)
        return x

    @staticmethod
    def init_weight(m):
        if isinstance(m, Linear):
            init.normal_(m.weight, std=.02)
            if isinstance(m, Linear) and m.bias is not None:
                init.constant_(m.bias, 0)
        elif isinstance(m, LayerNorm):
            init.constant_(m.bias, 0)
            init.constant_(m.weight, 1.0)

    @staticmethod
    def sinusoidal_embedding(n_channels, dim):
        pe = torch.FloatTensor([[p / (10000 ** (2 * (i // 2) / dim)) for i in range(dim)]
                                for p in range(n_channels)])
        pe[:, 0::2] = torch.sin(pe[:, 0::2])
        pe[:, 1::2] = torch.cos(pe[:, 1::2])
        return pe.unsqueeze(0)

# ViT


In [43]:
from torch.hub import load_state_dict_from_url
import torch.nn as nn

try:
    from timm.models.registry import register_model
except ImportError:
    pass

model_urls = {
}


class ViTLite(nn.Module):
    def __init__(self,
                 img_size=224,
                 embedding_dim=768,
                 n_input_channels=3,
                 kernel_size=16,
                 dropout=0.,
                 attention_dropout=0.1,
                 stochastic_depth=0.1,
                 num_layers=14,
                 num_heads=6,
                 mlp_ratio=4.0,
                 num_classes=1000,
                 positional_embedding='learnable',
                 *args, **kwargs):
        super(ViTLite, self).__init__()
        assert img_size % kernel_size == 0, f"Image size ({img_size}) has to be" \
                                            f"divisible by patch size ({kernel_size})"
        self.tokenizer = Tokenizer(n_input_channels=n_input_channels,
                                   n_output_channels=embedding_dim,
                                   kernel_size=kernel_size,
                                   stride=kernel_size,
                                   padding=0,
                                   max_pool=False,
                                   activation=None,
                                   n_conv_layers=1,
                                   conv_bias=True)

        self.classifier = TransformerClassifier(
            sequence_length=self.tokenizer.sequence_length(n_channels=n_input_channels,
                                                           height=img_size,
                                                           width=img_size),
            embedding_dim=embedding_dim,
            seq_pool=False,
            dropout=dropout,
            attention_dropout=attention_dropout,
            stochastic_depth=stochastic_depth,
            num_layers=num_layers,
            num_heads=num_heads,
            mlp_ratio=mlp_ratio,
            num_classes=num_classes,
            positional_embedding=positional_embedding
        )

    def forward(self, x):
        x = self.tokenizer(x)
        return self.classifier(x)


def _vit_lite(arch, pretrained, progress,
              num_layers, num_heads, mlp_ratio, embedding_dim,
              kernel_size=4, *args, **kwargs):
    model = ViTLite(num_layers=num_layers,
                    num_heads=num_heads,
                    mlp_ratio=mlp_ratio,
                    embedding_dim=embedding_dim,
                    kernel_size=kernel_size,
                    positional_embedding='learnable',
                    *args, **kwargs)

    if pretrained and arch in model_urls:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        if positional_embedding == 'learnable':
            state_dict = pe_check(model, state_dict)
        elif positional_embedding == 'sine':
            state_dict['classifier.positional_emb'] = model.state_dict()['classifier.positional_emb']
        state_dict = pe_check(model, state_dict)
        model.load_state_dict(state_dict)
    return model


def vit_2(*args, **kwargs):
    return _vit_lite(num_layers=2, num_heads=2, mlp_ratio=1, embedding_dim=128,
                     *args, **kwargs)


def vit_4(*args, **kwargs):
    return _vit_lite(num_layers=4, num_heads=2, mlp_ratio=1, embedding_dim=128,
                     *args, **kwargs)


def vit_6(*args, **kwargs):
    return _vit_lite(num_layers=6, num_heads=4, mlp_ratio=2, embedding_dim=256,
                     *args, **kwargs)


def vit_7(*args, **kwargs):
    return _vit_lite(num_layers=7, num_heads=4, mlp_ratio=2, embedding_dim=256,
                     *args, **kwargs)


@register_model
def vit_2_4_32(pretrained=False, progress=False,
               img_size=32, positional_embedding='learnable', num_classes=10,
               *args, **kwargs):
    return vit_2('vit_2_4_32', pretrained, progress,
                 kernel_size=4,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def vit_2_4_32_sine(pretrained=False, progress=False,
                    img_size=32, positional_embedding='sine', num_classes=10,
                    *args, **kwargs):
    return vit_2('vit_2_4_32_sine', pretrained, progress,
                 kernel_size=4,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def vit_4_4_32(pretrained=False, progress=False,
               img_size=32, positional_embedding='learnable', num_classes=10,
               *args, **kwargs):
    return vit_4('vit_4_4_32', pretrained, progress,
                 kernel_size=4,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def vit_4_4_32_sine(pretrained=False, progress=False,
                    img_size=32, positional_embedding='sine', num_classes=10,
                    *args, **kwargs):
    return vit_4('vit_4_4_32_sine', pretrained, progress,
                 kernel_size=4,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def vit_6_4_32(pretrained=False, progress=False,
               img_size=32, positional_embedding='learnable', num_classes=10,
               *args, **kwargs):
    return vit_6('vit_6_4_32', pretrained, progress,
                 kernel_size=4,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def vit_6_4_32_sine(pretrained=False, progress=False,
                    img_size=32, positional_embedding='sine', num_classes=10,
                    *args, **kwargs):
    return vit_6('vit_6_4_32_sine', pretrained, progress,
                 kernel_size=4,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def vit_7_4_32(pretrained=False, progress=False,
               img_size=32, positional_embedding='learnable', num_classes=10,
               *args, **kwargs):
    return vit_7('vit_7_4_32', pretrained, progress,
                 kernel_size=4,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def vit_7_4_32_sine(pretrained=False, progress=False,
                    img_size=32, positional_embedding='sine', num_classes=10,
                    *args, **kwargs):
    return vit_7('vit_7_4_32_sine', pretrained, progress,
                 kernel_size=4,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


# CVT

In [44]:
from torch.hub import load_state_dict_from_url
import torch.nn as nn

try:
    from timm.models.registry import register_model
except ImportError:
    pass

model_urls = {
}


class CVT(nn.Module):
    def __init__(self,
                 img_size=224,
                 embedding_dim=768,
                 n_input_channels=3,
                 kernel_size=16,
                 dropout=0.,
                 attention_dropout=0.1,
                 stochastic_depth=0.1,
                 num_layers=14,
                 num_heads=6,
                 mlp_ratio=4.0,
                 num_classes=1000,
                 positional_embedding='learnable',
                 *args, **kwargs):
        super(CVT, self).__init__()
        assert img_size % kernel_size == 0, f"Image size ({img_size}) has to be" \
                                            f"divisible by patch size ({kernel_size})"
        self.tokenizer = Tokenizer(n_input_channels=n_input_channels,
                                   n_output_channels=embedding_dim,
                                   kernel_size=kernel_size,
                                   stride=kernel_size,
                                   padding=0,
                                   max_pool=False,
                                   activation=None,
                                   n_conv_layers=1,
                                   conv_bias=True)

        self.classifier = TransformerClassifier(
            sequence_length=self.tokenizer.sequence_length(n_channels=n_input_channels,
                                                           height=img_size,
                                                           width=img_size),
            embedding_dim=embedding_dim,
            seq_pool=True,
            dropout=dropout,
            attention_dropout=attention_dropout,
            stochastic_depth=stochastic_depth,
            num_layers=num_layers,
            num_heads=num_heads,
            mlp_ratio=mlp_ratio,
            num_classes=num_classes,
            positional_embedding=positional_embedding
        )

    def forward(self, x):
        x = self.tokenizer(x)
        return self.classifier(x)


def _cvt(arch, pretrained, progress,
         num_layers, num_heads, mlp_ratio, embedding_dim,
         kernel_size=4, positional_embedding='learnable',
         *args, **kwargs):
    model = CVT(num_layers=num_layers,
                num_heads=num_heads,
                mlp_ratio=mlp_ratio,
                embedding_dim=embedding_dim,
                kernel_size=kernel_size,
                *args, **kwargs)

    if pretrained and arch in model_urls:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        if positional_embedding == 'learnable':
            state_dict = pe_check(model, state_dict)
        elif positional_embedding == 'sine':
            state_dict['classifier.positional_emb'] = model.state_dict()['classifier.positional_emb']
        model.load_state_dict(state_dict)
    return model

def cvt_2(*args, **kwargs):
    return _cvt(num_layers=2, num_heads=2, mlp_ratio=1, embedding_dim=128,
                *args, **kwargs)


def cvt_4(*args, **kwargs):
    return _cvt(num_layers=4, num_heads=2, mlp_ratio=1, embedding_dim=128,
                *args, **kwargs)


def cvt_6(*args, **kwargs):
    return _cvt(num_layers=6, num_heads=4, mlp_ratio=2, embedding_dim=256,
                *args, **kwargs)


def cvt_7(*args, **kwargs):
    return _cvt(num_layers=7, num_heads=4, mlp_ratio=2, embedding_dim=256,
                *args, **kwargs)


def cvt_8(*args, **kwargs):
    return _cvt(num_layers=8, num_heads=4, mlp_ratio=2, embedding_dim=256,
                *args, **kwargs)


@register_model
def cvt_2_4_32(pretrained=False, progress=False,
               img_size=32, positional_embedding='learnable', num_classes=10,
               *args, **kwargs):
    return cvt_2('cvt_2_4_32', pretrained, progress,
                 kernel_size=4,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cvt_2_4_32_sine(pretrained=False, progress=False,
                    img_size=32, positional_embedding='sine', num_classes=10,
                    *args, **kwargs):
    return cvt_2('cvt_2_4_32_sine', pretrained, progress,
                 kernel_size=4,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cvt_4_4_32(pretrained=False, progress=False,
               img_size=32, positional_embedding='learnable', num_classes=10,
               *args, **kwargs):
    return cvt_4('cvt_4_4_32', pretrained, progress,
                 kernel_size=4,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cvt_4_4_32_sine(pretrained=False, progress=False,
                    img_size=32, positional_embedding='sine', num_classes=10,
                    *args, **kwargs):
    return cvt_4('cvt_4_4_32_sine', pretrained, progress,
                 kernel_size=4,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cvt_6_4_32(pretrained=False, progress=False,
               img_size=32, positional_embedding='learnable', num_classes=10,
               *args, **kwargs):
    return cvt_6('cvt_6_4_32', pretrained, progress,
                 kernel_size=4,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cvt_6_4_32_sine(pretrained=False, progress=False,
                    img_size=32, positional_embedding='sine', num_classes=10,
                    *args, **kwargs):
    return cvt_6('cvt_6_4_32_sine', pretrained, progress,
                 kernel_size=4,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cvt_7_4_32(pretrained=False, progress=False,
               img_size=32, positional_embedding='learnable', num_classes=10,
               *args, **kwargs):
    return cvt_7('cvt_7_4_32', pretrained, progress,
                 kernel_size=4,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cvt_7_4_32_sine(pretrained=False, progress=False,
                    img_size=32, positional_embedding='sine', num_classes=10,
                    *args, **kwargs):
    return cvt_7('cvt_7_4_32_sine', pretrained, progress,
                 kernel_size=4,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)

# CCT

In [45]:
%%writefile cct.py


from torch.hub import load_state_dict_from_url
import torch.nn as nn
from tokenizer import Tokenizer
from transformers_component import TransformerEncoderLayer, TransformerClassifier
from helpers import pe_check, fc_check

try:
    from timm.models.registry import register_model
except ImportError:
    pass

model_urls = {
    'cct_7_3x1_32':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_cifar10_300epochs.pth',
    'cct_7_3x1_32_sine':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_sine_cifar10_5000epochs.pth',
    'cct_7_3x1_32_c100':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_cifar100_300epochs.pth',
    'cct_7_3x1_32_sine_c100':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_sine_cifar100_5000epochs.pth',
    'cct_7_7x2_224_sine':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_7x2_224_flowers102.pth',
    'cct_14_7x2_224':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_14_7x2_224_imagenet.pth',
    'cct_14_7x2_384':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/finetuned/cct_14_7x2_384_imagenet.pth',
    'cct_14_7x2_384_fl':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/finetuned/cct_14_7x2_384_flowers102.pth',
}


class CCT(nn.Module):
    def __init__(self,
                 img_size=224,
                 embedding_dim=768,
                 n_input_channels=3,
                 n_conv_layers=1,
                 kernel_size=7,
                 stride=2,
                 padding=3,
                 pooling_kernel_size=3,
                 pooling_stride=2,
                 pooling_padding=1,
                 dropout=0.,
                 attention_dropout=0.1,
                 stochastic_depth=0.1,
                 num_layers=14,
                 num_heads=6,
                 mlp_ratio=4.0,
                 num_classes=1000,
                 positional_embedding='learnable',
                 *args, **kwargs):
        super(CCT, self).__init__()

        self.tokenizer = Tokenizer(n_input_channels=n_input_channels,
                                   n_output_channels=embedding_dim,
                                   kernel_size=kernel_size,
                                   stride=stride,
                                   padding=padding,
                                   pooling_kernel_size=pooling_kernel_size,
                                   pooling_stride=pooling_stride,
                                   pooling_padding=pooling_padding,
                                   max_pool=True,
                                   activation=nn.ReLU,
                                   n_conv_layers=n_conv_layers,
                                   conv_bias=False)

        self.classifier = TransformerClassifier(
            sequence_length=self.tokenizer.sequence_length(n_channels=n_input_channels,
                                                           height=img_size,
                                                           width=img_size),
            embedding_dim=embedding_dim,
            seq_pool=True,
            dropout=dropout,
            attention_dropout=attention_dropout,
            stochastic_depth=stochastic_depth,
            num_layers=num_layers,
            num_heads=num_heads,
            mlp_ratio=mlp_ratio,
            num_classes=num_classes,
            positional_embedding=positional_embedding
        )

    def forward(self, x):
        x = self.tokenizer(x)
        return self.classifier(x)


def _cct(arch, pretrained, progress,
         num_layers, num_heads, mlp_ratio, embedding_dim,
         kernel_size=3, stride=None, padding=None,
         positional_embedding='learnable',
         *args, **kwargs):
    stride = stride if stride is not None else max(1, (kernel_size // 2) - 1)
    padding = padding if padding is not None else max(1, (kernel_size // 2))
    model = CCT(num_layers=num_layers,
                num_heads=num_heads,
                mlp_ratio=mlp_ratio,
                embedding_dim=embedding_dim,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
                *args, **kwargs)

    if pretrained:
        if arch in model_urls:
            state_dict = load_state_dict_from_url(model_urls[arch],
                                                  progress=progress)
            if positional_embedding == 'learnable':
                state_dict = pe_check(model, state_dict)
            elif positional_embedding == 'sine':
                state_dict['classifier.positional_emb'] = model.state_dict()['classifier.positional_emb']
            state_dict = fc_check(model, state_dict)
            model.load_state_dict(state_dict)
        else:
            raise RuntimeError(f'Variant {arch} does not yet have pretrained weights.')
    return model


def cct_2(arch, pretrained, progress, *args, **kwargs):
    return _cct(arch, pretrained, progress, num_layers=2, num_heads=2, mlp_ratio=1, embedding_dim=128,
                *args, **kwargs)


def cct_4(arch, pretrained, progress, *args, **kwargs):
    return _cct(arch, pretrained, progress, num_layers=4, num_heads=2, mlp_ratio=1, embedding_dim=128,
                *args, **kwargs)


def cct_6(arch, pretrained, progress, *args, **kwargs):
    return _cct(arch, pretrained, progress, num_layers=6, num_heads=4, mlp_ratio=2, embedding_dim=256,
                *args, **kwargs)


def cct_7(arch, pretrained, progress, *args, **kwargs):
    return _cct(arch, pretrained, progress, num_layers=7, num_heads=4, mlp_ratio=2, embedding_dim=256,
                *args, **kwargs)


def cct_14(arch, pretrained, progress, *args, **kwargs):
    return _cct(arch, pretrained, progress, num_layers=14, num_heads=6, mlp_ratio=3, embedding_dim=384,
                *args, **kwargs)

@register_model
def cct_2_3x2_32(pretrained=False, progress=False,
                 img_size=32, positional_embedding='learnable', num_classes=10,
                 *args, **kwargs):
    return cct_2('cct_2_3x2_32', pretrained, progress,
                 kernel_size=3, n_conv_layers=2,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cct_2_3x2_32_sine(pretrained=False, progress=False,
                      img_size=32, positional_embedding='sine', num_classes=10,
                      *args, **kwargs):
    return cct_2('cct_2_3x2_32_sine', pretrained, progress,
                 kernel_size=3, n_conv_layers=2,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cct_4_3x2_32(pretrained=False, progress=False,
                 img_size=32, positional_embedding='learnable', num_classes=10,
                 *args, **kwargs):
    return cct_4('cct_4_3x2_32', pretrained, progress,
                 kernel_size=3, n_conv_layers=2,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cct_4_3x2_32_sine(pretrained=False, progress=False,
                      img_size=32, positional_embedding='sine', num_classes=10,
                      *args, **kwargs):
    return cct_4('cct_4_3x2_32_sine', pretrained, progress,
                 kernel_size=3, n_conv_layers=2,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cct_6_3x1_32(pretrained=False, progress=False,
                 img_size=32, positional_embedding='learnable', num_classes=10,
                 *args, **kwargs):
    return cct_6('cct_6_3x1_32', pretrained, progress,
                 kernel_size=3, n_conv_layers=1,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cct_6_3x1_32_sine(pretrained=False, progress=False,
                      img_size=32, positional_embedding='sine', num_classes=10,
                      *args, **kwargs):
    return cct_6('cct_6_3x1_32_sine', pretrained, progress,
                 kernel_size=3, n_conv_layers=1,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cct_6_3x2_32(pretrained=False, progress=False,
                 img_size=32, positional_embedding='learnable', num_classes=10,
                 *args, **kwargs):
    return cct_6('cct_6_3x2_32', pretrained, progress,
                 kernel_size=3, n_conv_layers=2,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cct_6_3x2_32_sine(pretrained=False, progress=False,
                      img_size=32, positional_embedding='sine', num_classes=10,
                      *args, **kwargs):
    return cct_6('cct_6_3x2_32_sine', pretrained, progress,
                 kernel_size=3, n_conv_layers=2,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cct_7_3x1_32(pretrained=False, progress=False,
                 img_size=32, positional_embedding='learnable', num_classes=10,
                 *args, **kwargs):
    return cct_7('cct_7_3x1_32', pretrained, progress,
                 kernel_size=3, n_conv_layers=1,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)

@register_model
def cct_7_3x1_32_sine(pretrained=False, progress=False,
                      img_size=32, positional_embedding='sine', num_classes=10,
                      *args, **kwargs):
    return cct_7('cct_7_3x1_32_sine', pretrained, progress,
                 kernel_size=3, n_conv_layers=1,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cct_7_3x1_32_c100(pretrained=False, progress=False,
                      img_size=32, positional_embedding='learnable', num_classes=100,
                      *args, **kwargs):
    return cct_7('cct_7_3x1_32_c100', pretrained, progress,
                 kernel_size=3, n_conv_layers=1,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cct_7_3x1_32_sine_c100(pretrained=False, progress=False,
                           img_size=32, positional_embedding='sine', num_classes=100,
                           *args, **kwargs):
    return cct_7('cct_7_3x1_32_sine_c100', pretrained, progress,
                 kernel_size=3, n_conv_layers=1,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cct_7_3x2_32(pretrained=False, progress=False,
                 img_size=32, positional_embedding='learnable', num_classes=10,
                 *args, **kwargs):
    return cct_7('cct_7_3x2_32', pretrained, progress,
                 kernel_size=3, n_conv_layers=2,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cct_7_3x2_32_sine(pretrained=False, progress=False,
                      img_size=32, positional_embedding='sine', num_classes=10,
                      *args, **kwargs):
    return cct_7('cct_7_3x2_32_sine', pretrained, progress,
                 kernel_size=3, n_conv_layers=2,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cct_7_7x2_224(pretrained=False, progress=False,
                  img_size=224, positional_embedding='learnable', num_classes=102,
                  *args, **kwargs):
    return cct_7('cct_7_7x2_224', pretrained, progress,
                 kernel_size=7, n_conv_layers=2,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cct_7_7x2_224_sine(pretrained=False, progress=False,
                       img_size=224, positional_embedding='sine', num_classes=102,
                       *args, **kwargs):
    return cct_7('cct_7_7x2_224_sine', pretrained, progress,
                 kernel_size=7, n_conv_layers=2,
                 img_size=img_size, positional_embedding=positional_embedding,
                 num_classes=num_classes,
                 *args, **kwargs)


@register_model
def cct_14_7x2_224(pretrained=False, progress=False,
                   img_size=224, positional_embedding='learnable', num_classes=1000,
                   *args, **kwargs):
    return cct_14('cct_14_7x2_224', pretrained, progress,
                  kernel_size=7, n_conv_layers=2,
                  img_size=img_size, positional_embedding=positional_embedding,
                  num_classes=num_classes,
                  *args, **kwargs)


@register_model
def cct_14_7x2_384(pretrained=False, progress=False,
                   img_size=384, positional_embedding='learnable', num_classes=1000,
                   *args, **kwargs):
    return cct_14('cct_14_7x2_384', pretrained, progress,
                  kernel_size=7, n_conv_layers=2,
                  img_size=img_size, positional_embedding=positional_embedding,
                  num_classes=num_classes,
                  *args, **kwargs)


@register_model
def cct_14_7x2_384_fl(pretrained=False, progress=False,
                      img_size=384, positional_embedding='learnable', num_classes=102,
                      *args, **kwargs):
    return cct_14('cct_14_7x2_384_fl', pretrained, progress,
                  kernel_size=7, n_conv_layers=2,
                  img_size=img_size, positional_embedding=positional_embedding,
                  num_classes=num_classes,
                  *args, **kwargs)

Overwriting cct.py


# MCVT

In [46]:
import torch.nn as nn

try:
    from timm.models.registry import register_model
except ImportError:
    pass


class MemoryCVT(nn.Module):
    def __init__(self,
                 img_size=224,
                 embedding_dim=768,
                 n_input_channels=3,
                 width_patches=2,
                 dropout=0.,
                 attention_dropout=0.1,
                 stochastic_depth=0.1,
                 num_layers=14,
                 num_heads=6,
                 mlp_ratio=4.0,
                 num_classes=1000,
                 positional_embedding='learnable',
                 *args, **kwargs):
        super(MemoryCVT, self).__init__()
        assert img_size % width_patches == 0, f"Image size ({img_size}) has to be" \
                                              f"divisible by patch size ({width_patches})"
        self.tokenizer = VerticalTokenizer(img_size=img_size, channels=n_input_channels, dim=embedding_dim,
                                           width_patches=width_patches)

        self.classifier = TransformerMemoryClassifier(
            sequence_length=self.tokenizer.sequence_length(n_channels=n_input_channels,
                                                           height=img_size,
                                                           width=img_size),
            embedding_dim=embedding_dim,
            dropout=dropout,
            attention_dropout=attention_dropout,
            stochastic_depth=stochastic_depth,
            num_layers=num_layers,
            num_heads=num_heads,
            mlp_ratio=mlp_ratio,
            num_classes=num_classes,
            positional_embedding=positional_embedding
        )

    def forward(self, x):
        x = self.tokenizer(x)
        return self.classifier(x)


def _memory_cvt(num_layers, num_heads, mlp_ratio, embedding_dim,
                width_patches=2,
                *args, **kwargs):
    model = MemoryCVT(num_layers=num_layers,
                      num_heads=num_heads,
                      mlp_ratio=mlp_ratio,
                      embedding_dim=embedding_dim,
                      width_patches=width_patches,
                      *args, **kwargs)
    return model


def memory_cvt_4(*args, **kwargs):
    return _memory_cvt(num_layers=4, num_heads=2, mlp_ratio=1, embedding_dim=128,
                       *args, **kwargs)


def memory_cvt_7(*args, **kwargs):
    return _memory_cvt(num_layers=7, num_heads=4, mlp_ratio=2, embedding_dim=256,
                       *args, **kwargs)


@register_model
def mcvt_4_1_32(
        img_size=32, positional_embedding='learnable', num_classes=10,
        *args, **kwargs):
    return memory_cvt_4(arch='mcvt_4_1_32',
                        width_patches=1,
                        img_size=img_size, positional_embedding=positional_embedding,
                        num_classes=num_classes,
                        *args, **kwargs)


@register_model
def mcvt_4_2_32(img_size=32, positional_embedding='learnable', num_classes=10,
                *args, **kwargs):
    return memory_cvt_4(arch='mcvt_4_2_32',
                        width_patches=2,
                        img_size=img_size, positional_embedding=positional_embedding,
                        num_classes=num_classes,
                        *args, **kwargs)


@register_model
def mcvt_7_1_32(img_size=32, positional_embedding='learnable', num_classes=10,
                *args, **kwargs):
    return memory_cvt_7(arch='mcvt_7_1_32',
                        width_patches=1,
                        img_size=img_size, positional_embedding=positional_embedding,
                        num_classes=num_classes,
                        *args, **kwargs)


@register_model
def mcvt_7_2_32(img_size=32, positional_embedding='learnable', num_classes=10,
                *args, **kwargs):
    return memory_cvt_7(arch='mcvt_7_2_32',
                        width_patches=2,
                        img_size=img_size, positional_embedding=positional_embedding,
                        num_classes=num_classes,
                        *args, **kwargs)


# MCCT

In [47]:
import torch.nn as nn
from torch.hub import load_state_dict_from_url
import torch
from torch.nn import Module, ModuleList, Linear, Dropout, LayerNorm, Parameter, init, LSTM
import torch.nn.functional as F

try:
    from timm.models.registry import register_model
except ImportError:
    pass


model_urls = {
    'cct_7_3x1_32':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_cifar10_300epochs.pth',
    'cct_7_3x1_32_sine':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_sine_cifar10_5000epochs.pth',
    'cct_7_3x1_32_c100':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_cifar100_300epochs.pth',
    'cct_7_3x1_32_sine_c100':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_sine_cifar100_5000epochs.pth',
    'cct_7_7x2_224_sine':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_7x2_224_flowers102.pth',
    'cct_14_7x2_224':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_14_7x2_224_imagenet.pth',
    'cct_14_7x2_384':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/finetuned/cct_14_7x2_384_imagenet.pth',
    'cct_14_7x2_384_fl':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/finetuned/cct_14_7x2_384_flowers102.pth',
}


class MemoryCCT(nn.Module):
    def __init__(self,
                 img_size=224,
                 embedding_dim=768,
                 n_input_channels=3,
                 n_conv_layers=1,
                 kernel_size=7,
                 stride=2,
                 padding=3,
                 pooling_kernel_size=3,
                 pooling_stride=2,
                 pooling_padding=1,
                 dropout=0.,
                 attention_dropout=0.1,
                 stochastic_depth=0.1,
                 num_layers=14,
                 num_heads=6,
                 mlp_ratio=4.0,
                 num_classes=1000,
                 positional_embedding='learnable',
                 *args, **kwargs):
        super(MemoryCCT, self).__init__()

        self.tokenizer = Tokenizer(n_input_channels=n_input_channels,
                                   n_output_channels=embedding_dim,
                                   kernel_size=kernel_size,
                                   stride=stride,
                                   padding=padding,
                                   pooling_kernel_size=pooling_kernel_size,
                                   pooling_stride=pooling_stride,
                                   pooling_padding=pooling_padding,
                                   max_pool=True,
                                   activation=nn.ReLU,
                                   n_conv_layers=n_conv_layers,
                                   conv_bias=False)

        self.classifier = TransformerMemoryClassifier(
            sequence_length=self.tokenizer.sequence_length(n_channels=n_input_channels,
                                                           height=img_size,
                                                           width=img_size),
            embedding_dim=embedding_dim,
            dropout=dropout,
            attention_dropout=attention_dropout,
            stochastic_depth=stochastic_depth,
            num_layers=num_layers,
            num_heads=num_heads,
            mlp_ratio=mlp_ratio,
            num_classes=num_classes,
            positional_embedding=positional_embedding
        )

    def forward(self, x):
        x = self.tokenizer(x)
        return self.classifier(x)


    def extract_embedding(self, x):
        x = self.tokenizer(x)
        return self.classifier.extract_embedding(x)

    def extract_memory_embedding(self, x):
        x = self.tokenizer(x)
        return self.classifier.extract_memory_embedding(x)


def _memory_cct(arch, pretrained, progress, pretrained_arch,
                num_layers, num_heads, mlp_ratio, embedding_dim,
                kernel_size=3, stride=None, padding=None, positional_embedding='learnable',
                *args, **kwargs):
    stride = stride if stride is not None else max(1, (kernel_size // 2) - 1)
    padding = padding if padding is not None else max(1, (kernel_size // 2))
    model = MemoryCCT(num_layers=num_layers,
                      num_heads=num_heads,
                      mlp_ratio=mlp_ratio,
                      embedding_dim=embedding_dim,
                      kernel_size=kernel_size,
                      stride=stride,
                      padding=padding,
                      *args, **kwargs)

    if pretrained:
        if pretrained_arch in model_urls:
            state_dict = load_state_dict_from_url(model_urls[pretrained_arch],
                                                  progress=progress)
            if positional_embedding == 'learnable':
                state_dict = pe_check(model, state_dict)
            elif positional_embedding == 'sine':
                state_dict['classifier.positional_emb'] = model.state_dict()['classifier.positional_emb']

            model_dict = model.state_dict()

            state_dict = {k: v for k, v in state_dict.items() if
                                k in model_dict and v.size() == model_dict[k].size()}
            model_dict.update(state_dict)
            model.load_state_dict(state_dict, strict=False)
        else:
            raise RuntimeError(f'Variant {pretrained_arch} does not yet have pretrained weights.')

    return model


def memory_cct_7(arch, pretrained, progress, *args, **kwargs):
    return _memory_cct(arch=arch, pretrained=pretrained, progress=progress,
                       num_layers=7, num_heads=4, mlp_ratio=2, embedding_dim=256,
                       *args, **kwargs)


def memory_cct_4(arch, pretrained, progress, *args, **kwargs):
    return _memory_cct(arch=arch, pretrained=pretrained, progress=progress,
                       num_layers=4, num_heads=4, mlp_ratio=2, embedding_dim=256,
                       *args, **kwargs)


@register_model
def mcct_4_3x1_32(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                  *args, **kwargs):
    return memory_cct_4(arch='mcct_4_3x1_32', pretrained=pretrained, progress=progress,
                        kernel_size=3, n_conv_layers=1,  pretrained_arch="",
                        img_size=img_size, positional_embedding=positional_embedding,
                        num_classes=num_classes,
                        *args, **kwargs)

@register_model
def mcct_7_3x1_32(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                  *args, **kwargs):
    return memory_cct_7(arch='mcct_7_3x1_32', pretrained=pretrained, progress=progress, pretrained_arch="cct_7_3x1_32",
                        kernel_size=3, n_conv_layers=1,
                        img_size=img_size, positional_embedding=positional_embedding,
                        num_classes=num_classes,
                        *args, **kwargs)

@register_model
def mcct_7_3x1_32_c100(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                  *args, **kwargs):
    return memory_cct_7(arch='mcct_7_3x1_32_c100', pretrained=pretrained, progress=progress, pretrained_arch="cct_7_3x1_32_c100",
                        kernel_size=3, n_conv_layers=1,
                        img_size=img_size, positional_embedding=positional_embedding,
                        num_classes=num_classes,
                        *args, **kwargs)




# P_MCCT

In [48]:
import torch.nn as nn
from torch.hub import load_state_dict_from_url

import torch
from torch.nn import Module, ModuleList, Linear, Dropout, LayerNorm, Parameter, init, LSTM
import torch.nn.functional as F

try:
    from timm.models.registry import register_model
except ImportError:
    pass

model_urls = {
    'cct_7_3x1_32':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_cifar10_300epochs.pth',
    'cct_7_3x1_32_sine':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_sine_cifar10_5000epochs.pth',
    'cct_7_3x1_32_c100':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_cifar100_300epochs.pth',
    'cct_7_3x1_32_sine_c100':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_sine_cifar100_5000epochs.pth',
    'cct_7_7x2_224_sine':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_7x2_224_flowers102.pth',
    'cct_14_7x2_224':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_14_7x2_224_imagenet.pth',
    'cct_14_7x2_384':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/finetuned/cct_14_7x2_384_imagenet.pth',
    'cct_14_7x2_384_fl':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/finetuned/cct_14_7x2_384_flowers102.pth',
}


class ParallelMemoryCCT(nn.Module):
    def __init__(self,
                 img_size=224,
                 embedding_dim=768,
                 n_input_channels=3,
                 n_conv_layers=1,
                 kernel_size=7,
                 stride=2,
                 padding=3,
                 pooling_kernel_size=3,
                 pooling_stride=2,
                 pooling_padding=1,
                 dropout=0.,
                 attention_dropout=0.1,
                 stochastic_depth=0.1,
                 num_layers=14,
                 num_heads=6,
                 mlp_ratio=4.0,
                 num_classes=1000,
                 positional_embedding='learnable',
                 width_patches_lstm=2,
                 type='concatenate',  # type of union lstm output with cct output: concatenate | sum | average
                 *args, **kwargs):
        super(ParallelMemoryCCT, self).__init__()

        self.tokenizer_cct = Tokenizer(n_input_channels=n_input_channels,
                                       n_output_channels=embedding_dim,
                                       kernel_size=kernel_size,
                                       stride=stride,
                                       padding=padding,
                                       pooling_kernel_size=pooling_kernel_size,
                                       pooling_stride=pooling_stride,
                                       pooling_padding=pooling_padding,
                                       max_pool=True,
                                       activation=nn.ReLU,
                                       n_conv_layers=n_conv_layers,
                                       conv_bias=False)

        self.tokenizer_lstm = VerticalTokenizer(img_size=img_size, channels=n_input_channels, dim=embedding_dim,
                                                width_patches=width_patches_lstm)

        self.classifier = ParallelMemoryClassifier(
            sequence_length=self.tokenizer_cct.sequence_length(n_channels=n_input_channels,
                                                               height=img_size,
                                                               width=img_size),
            embedding_dim=embedding_dim,
            dropout=dropout,
            attention_dropout=attention_dropout,
            stochastic_depth=stochastic_depth,
            num_layers=num_layers,
            num_heads=num_heads,
            mlp_ratio=mlp_ratio,
            num_classes=num_classes,
            positional_embedding=positional_embedding,
            type=type
        )

    def forward(self, x):
        x_cct = self.tokenizer_cct(x)
        x_lstm = self.tokenizer_lstm(x)

        return self.classifier(x_cct, x_lstm)



def _parallel_memory_cct(arch, pretrained, progress, pretrained_arch, num_layers, num_heads, mlp_ratio, embedding_dim,
                         kernel_size=3, stride=None, padding=None, positional_embedding='learnable',
                         width_patches=2, type='concatenate',
                         *args, **kwargs):
    stride = stride if stride is not None else max(1, (kernel_size // 2) - 1)
    padding = padding if padding is not None else max(1, (kernel_size // 2))
    model = ParallelMemoryCCT(num_layers=num_layers,
                              num_heads=num_heads,
                              mlp_ratio=mlp_ratio,
                              embedding_dim=embedding_dim,
                              kernel_size=kernel_size,
                              stride=stride,
                              padding=padding, width_patches_lstm=width_patches,
                              type=type,
                              *args, **kwargs)

    if pretrained:
        if pretrained_arch in model_urls:
            state_dict = load_state_dict_from_url(model_urls[pretrained_arch],
                                                  progress=progress)
            if positional_embedding == 'learnable':
                state_dict = pe_check(model, state_dict)
            elif positional_embedding == 'sine':
                state_dict['classifier.positional_emb'] = model.state_dict()['classifier.positional_emb']

            model_dict = model.state_dict()

            state_dict = {k: v for k, v in state_dict.items() if
                                k in model_dict and v.size() == model_dict[k].size()}
            model_dict.update(state_dict)
            model.load_state_dict(state_dict, strict=False)
        else:
            raise RuntimeError(f'Variant {pretrained_arch} does not yet have pretrained weights.')

    return model


def parallel_memory_cct_4(arch, pretrained, progress, width_patches, type, *args, **kwargs):
    return _parallel_memory_cct(arch=arch, pretrained=pretrained, progress=progress,
                                num_layers=4, num_heads=2, mlp_ratio=1, embedding_dim=128,
                                width_patches=width_patches, type=type, *args, **kwargs)


def parallel_memory_cct_7(arch, pretrained, progress, width_patches, type, *args, **kwargs):
    return _parallel_memory_cct(arch=arch, pretrained=pretrained, progress=progress,
                                num_layers=7, num_heads=4, mlp_ratio=2, embedding_dim=256,
                                width_patches=width_patches, type=type, *args, **kwargs)


@register_model
def p_mcct_7_1_3x1_32(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                      *args, **kwargs):
    return parallel_memory_cct_7(arch='p_mcct_7_1_3x1_32', pretrained=pretrained, progress=progress,
                                 pretrained_arch="cct_7_3x1_32",
                                 kernel_size=3, n_conv_layers=1, width_patches=1,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='concatenate',
                                 *args, **kwargs)

@register_model
def p_mcct_7_1_3x1_32_c100(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                      *args, **kwargs):
    return parallel_memory_cct_7(arch='p_mcct_7_1_3x1_32_c100', pretrained=pretrained, progress=progress,
                                 pretrained_arch="cct_7_3x1_32_c100",
                                 kernel_size=3, n_conv_layers=1, width_patches=1,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='concatenate',
                                 *args, **kwargs)


@register_model
def p_mcct_7_1_3x1_32_sum(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                          *args, **kwargs):
    return parallel_memory_cct_7(arch='p_mcct_7_1_3x1_32_sum',
                                 pretrained=pretrained, progress=progress,
                                 pretrained_arch="cct_7_3x1_32",
                                 kernel_size=3, n_conv_layers=1, width_patches=1,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='sum',
                                 *args, **kwargs)


@register_model
def p_mcct_7_1_3x1_32_average(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                              *args, **kwargs):
    return parallel_memory_cct_7(arch='p_mcct_7_1_3x1_32_average', pretrained=pretrained, progress=progress,
                                 pretrained_arch="cct_7_3x1_32",
                                 kernel_size=3, n_conv_layers=1, width_patches=1,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='average',
                                 *args, **kwargs)


@register_model
def p_mcct_7_2_3x1_32(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                      *args, **kwargs):
    return parallel_memory_cct_7(arch='p_mcct_7_2_3x1_32', pretrained=pretrained, progress=progress,
                                 pretrained_arch="cct_7_3x1_32",
                                 kernel_size=3, n_conv_layers=1, width_patches=2,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='concatenate',
                                 *args, **kwargs)

@register_model
def p_mcct_7_2_3x1_32_c100(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                      *args, **kwargs):
    return parallel_memory_cct_7(arch='p_mcct_7_2_3x1_32_c100', pretrained=pretrained, progress=progress,
                                 pretrained_arch="cct_7_3x1_32_c100",
                                 kernel_size=3, n_conv_layers=1, width_patches=2,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='concatenate',
                                 *args, **kwargs)

@register_model
def p_mcct_7_2_3x1_32_sum(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                          *args, **kwargs):
    return parallel_memory_cct_7(arch='p_mcct_7_2_3x1_32_sum', pretrained=pretrained, progress=progress,
                                 pretrained_arch="cct_7_3x1_32",
                                 kernel_size=3, n_conv_layers=1, width_patches=2,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='sum',
                                 *args, **kwargs)


@register_model
def p_mcct_7_2_3x1_32_average(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                              *args, **kwargs):
    return parallel_memory_cct_7(arch='p_mcct_7_2_3x1_32_average', pretrained=pretrained, progress=progress,
                                 pretrained_arch="cct_7_3x1_32",
                                 kernel_size=3, n_conv_layers=1, width_patches=2,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='average',
                                 *args, **kwargs)


@register_model
def p_mcct_4_1_3x1_32(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                      *args, **kwargs):
    return parallel_memory_cct_4(arch='p_mcct_4_1_3x1_32', pretrained=pretrained, progress=progress,
                                 pretrained_arch="",
                                 kernel_size=3, n_conv_layers=1, width_patches=1,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='concatenate',
                                 *args, **kwargs)


@register_model
def p_mcct_4_1_3x1_32_sum(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                          *args, **kwargs):
    return parallel_memory_cct_4(arch='p_mcct_4_1_3x1_32_sum', pretrained=pretrained, progress=progress,
                                 pretrained_arch="",
                                 kernel_size=3, n_conv_layers=1, width_patches=1,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='sum',
                                 *args, **kwargs)


@register_model
def p_mcct_4_1_3x1_32_average(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                              *args, **kwargs):
    return parallel_memory_cct_4(arch='p_mcct_4_1_3x1_32_average', pretrained=pretrained, progress=progress,
                                 pretrained_arch="",
                                 kernel_size=3, n_conv_layers=1, width_patches=1,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='average',
                                 *args, **kwargs)


@register_model
def p_mcct_4_2_3x1_32(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                      *args, **kwargs):
    return parallel_memory_cct_4(arch='p_mcct_4_2_3x1_32', pretrained=pretrained, progress=progress,
                                 pretrained_arch="",
                                 kernel_size=3, n_conv_layers=1, width_patches=2,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='concatenate',
                                 *args, **kwargs)


@register_model
def p_mcct_4_2_3x1_32_sum(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                          *args, **kwargs):
    return parallel_memory_cct_4(arch='p_mcct_4_2_3x1_32_sum', pretrained=pretrained, progress=progress,
                                 pretrained_arch="",
                                 kernel_size=3, n_conv_layers=1, width_patches=2,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='sum',
                                 *args, **kwargs)


@register_model
def p_mcct_4_2_3x1_32_average(pretrained=False, progress=False, img_size=32, positional_embedding='learnable',
                              num_classes=10, *args, **kwargs):
    return parallel_memory_cct_4(arch='p_mcct_4_2_3x1_32_average', pretrained=pretrained, progress=progress,
                                 pretrained_arch="",
                                 kernel_size=3, n_conv_layers=1, width_patches=2,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='average',
                                 *args, **kwargs)

# Speaker CCT

In [49]:
import os

import torch.nn as nn
from timm.models.helpers import load_state_dict
from timm.models.layers import set_layer_config

import torch
from torch.nn import Linear, Module, ModuleList, Linear, Dropout, LayerNorm, Parameter, init, LSTM
import torch.nn.functional as F
#import wavencoder
import librosa
#from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
from torch.hub import load_state_dict_from_url
import numpy as np

try:
    from timm.models.registry import register_model, model_entrypoint
except ImportError:
    pass

model_urls = {
    'cct_7_3x1_32':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_cifar10_300epochs.pth',
    'cct_7_3x1_32_sine':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_sine_cifar10_5000epochs.pth',
    'cct_7_3x1_32_c100':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_cifar100_300epochs.pth',
    'cct_7_3x1_32_sine_c100':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_sine_cifar100_5000epochs.pth',
    'cct_7_7x2_224_sine':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_7x2_224_flowers102.pth',
    'cct_14_7x2_224':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_14_7x2_224_imagenet.pth',
    'cct_14_7x2_384':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/finetuned/cct_14_7x2_384_imagenet.pth',
    'cct_14_7x2_384_fl':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/finetuned/cct_14_7x2_384_flowers102.pth',
}

class SpeakerCCT(nn.Module):
    def __init__(self,
                 speaker_embedder=None,
                 gender_embedder=None,
                 corpus_embedder=None,
                 img_size=224,
                 embedding_dim=768,
                 n_input_channels=3,
                 n_conv_layers=1,
                 kernel_size=7,
                 stride=2,
                 padding=3,
                 pooling_kernel_size=3,
                 pooling_stride=2,
                 pooling_padding=1,
                 dropout=0.,
                 attention_dropout=0.1,
                 stochastic_depth=0.1,
                 num_layers=14,
                 num_heads=6,
                 mlp_ratio=4.0,
                 num_classes=1000,
                 positional_embedding='learnable',
                 type='self-attention',
                 n_layers_scnd_transformer=4,
                 n_heads_scnd_transformer=2,
                 after_fc_embedded=False,
                 *args, **kwargs):
        super(SpeakerCCT, self).__init__()

        self.tokenizer = Tokenizer(n_input_channels=n_input_channels,
                                   n_output_channels=embedding_dim,
                                   kernel_size=kernel_size,
                                   stride=stride,
                                   padding=padding,
                                   pooling_kernel_size=pooling_kernel_size,
                                   pooling_stride=pooling_stride,
                                   pooling_padding=pooling_padding,
                                   max_pool=True,
                                   activation=nn.ReLU,
                                   n_conv_layers=n_conv_layers,
                                   conv_bias=False)

        self.speaker_embedder = speaker_embedder
        self.gender_embedder = gender_embedder
        self.corpus_embedder = corpus_embedder

        if self.speaker_embedder is not None:
            self.speaker_embedder.requires_grad = False
        if self.gender_embedder is not None:
            self.gender_embedder.requires_grad = False
        if self.corpus_embedder is not None:
            self.corpus_embedder.requires_grad = False
        self.after_fc_embedded=after_fc_embedded

        self.classifier = SpeakerCCTClassifier(
            sequence_length=self.tokenizer.sequence_length(n_channels=n_input_channels,
                                                           height=img_size,
                                                           width=img_size),
            embedding_dim=embedding_dim,
            dropout=dropout,
            attention_dropout=attention_dropout,
            stochastic_depth=stochastic_depth,
            num_layers=num_layers,
            num_heads=num_heads,
            mlp_ratio=mlp_ratio,
            num_classes=num_classes,
            positional_embedding=positional_embedding,
            type=type,
            n_layers_scnd_transformer=n_layers_scnd_transformer,
            n_heads_scnd_transformer=n_heads_scnd_transformer,
            after_fc_embedded=after_fc_embedded
        )

    def forward(self, x):
        x_spect = self.tokenizer(x)
        x_speaker = None
        if self.speaker_embedder:
            if self.after_fc_embedded:
                x_speaker = self.speaker_embedder(x)
            else:
                x_speaker = self.speaker_embedder.extract_memory_embedding(x)
        x_gender = None
        if self.gender_embedder:
            if self.after_fc_embedded:
                x_gender = self.gender_embedder(x)
            else:
                x_gender = self.gender_embedder.extract_memory_embedding(x)
        x_corpus = None
        if self.corpus_embedder:
            if self.after_fc_embedded:
                x_corpus = self.corpus_embedder(x)
            else:
                x_corpus = self.corpus_embedder.extract_memory_embedding(x)
        return self.classifier(x_spect, x_speaker, x_gender, x_corpus)


class SpeakerCCTClassifier(Module):
    def __init__(self,
                 embedding_dim=768,
                 num_layers=12,
                 num_heads=12,
                 mlp_ratio=4.0,
                 num_classes=1000,
                 dropout=0.1,
                 attention_dropout=0.1,
                 stochastic_depth=0.1,
                 positional_embedding='learnable',
                 sequence_length=None,
                 type='self-attention',
                 n_layers_scnd_transformer=4,
                 n_heads_scnd_transformer=2,
                 after_fc_embedded=False
                 ):
        super().__init__()
        positional_embedding = positional_embedding if \
            positional_embedding in ['sine', 'learnable', 'none'] else 'sine'
        dim_feedforward = int(embedding_dim * mlp_ratio)
        self.embedding_dim = embedding_dim
        self.sequence_length = sequence_length
        self.num_tokens = 0
        self.num_classes = num_classes
        self.after_fc_embedded=after_fc_embedded

        assert sequence_length is not None or positional_embedding == 'none', \
            f"Positional embedding is set to {positional_embedding} and" \
            f" the sequence length was not specified."

        assert type == 'self-attention' or type == 'concatenate' or type == 'sum' or type == 'average', \
            f"Union output type is set to {type} that is not specified"
        self.union_type = type

        self.attention_pool_cct = Linear(self.embedding_dim, 1)
        #self.attention_pool_speaker = Linear(self.embedding_dim, 1)

        if positional_embedding != 'none':
            if positional_embedding == 'learnable':
                self.positional_emb = Parameter(torch.zeros(1, sequence_length, embedding_dim),
                                                requires_grad=True)
                init.normal_(self.positional_emb, std=0.2)
            else:
                self.positional_emb = Parameter(self.sinusoidal_embedding(sequence_length, embedding_dim),
                                                requires_grad=False)
        else:
            self.positional_emb = None

        self.dropout = Dropout(p=dropout)
        dpr = [x.item() for x in torch.linspace(0, stochastic_depth, num_layers)]
        self.blocks = ModuleList([
            TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads,
                                    dim_feedforward=dim_feedforward, dropout=dropout,
                                    attention_dropout=attention_dropout, drop_path_rate=dpr[i])
            for i in range(num_layers)])
        self.norm_cct = LayerNorm(embedding_dim)

        #self.speaker_encoder = wavencoder.models.Wav2Vec(pretrained=True)
        #self.fc_wav = Linear(5600, embedding_dim)
        self.lstm = LSTM(embedding_dim, embedding_dim)
        self.norm = LayerNorm(embedding_dim)

        fc_dim = embedding_dim

        if self.union_type == 'concatenate':
            fc_dim = embedding_dim * 2

        if self.after_fc_embedded:
            self.norm_spect = LayerNorm(fc_dim)
            self.fc_spect = Linear(fc_dim, num_classes)
            self.project_speaker = Linear(11, num_classes)
            self.project_gender = Linear(2, num_classes)
            self.project_corpus = Linear(5, num_classes)
            self.fc = Linear(num_classes, num_classes)
            if self.union_type == 'self-attention':
                """
                self.self_attn = Attention(dim=embedding_dim, num_heads=4,
                                               attention_dropout=attention_dropout, projection_dropout=dropout)
                """
                self.blocks_scnd_level = ModuleList([
                    TransformerEncoderLayer(d_model=num_classes, nhead=n_heads_scnd_transformer,
                                            dim_feedforward=dim_feedforward, dropout=dropout,
                                            attention_dropout=attention_dropout, drop_path_rate=dpr[i])
                    for i in range(n_layers_scnd_transformer)])
                self.positional_emb_scnd_level = Parameter(self.sinusoidal_embedding(3, embedding_dim),
                                                           requires_grad=False)
                self.norm_union = LayerNorm(num_classes)

        else:
            if self.union_type == 'self-attention':
                """
                self.self_attn = Attention(dim=embedding_dim, num_heads=4,
                                               attention_dropout=attention_dropout, projection_dropout=dropout)
                """
                self.blocks_scnd_level = ModuleList([
                    TransformerEncoderLayer(d_model=embedding_dim, nhead=n_heads_scnd_transformer,
                                            dim_feedforward=dim_feedforward, dropout=dropout,
                                            attention_dropout=attention_dropout, drop_path_rate=dpr[i])
                    for i in range(n_layers_scnd_transformer)])
                self.positional_emb_scnd_level = Parameter(self.sinusoidal_embedding(3, embedding_dim),
                                                           requires_grad=False)
                self.speaker_attention_pool = Linear(self.embedding_dim, 1)

            self.norm_union = LayerNorm(embedding_dim)
            self.fc = Linear(embedding_dim, num_classes)


        self.apply(self.init_weight)

    def forward(self, x_spect, x_speaker=None, x_gender=None, x_corpus=None):
        if self.positional_emb is None and x_spect.size(1) < self.sequence_length:
            x_spect = F.pad(x_spect, (0, 0, 0, self.n_channels - x_spect.size(1)), mode='constant', value=0)

        if self.positional_emb is not None:
            x_spect += self.positional_emb

        x_spect = self.dropout(x_spect)

        for blk in self.blocks:
            x_spect = blk(x_spect)
        #x_spect = self.lstm(x_spect)[0]
        x_spect = self.norm_cct(x_spect)
        x_spect = torch.matmul(F.softmax(self.attention_pool_cct(x_spect), dim=1).transpose(-1, -2), x_spect).squeeze(
            -2)

        """
        #x_speaker = self.speaker_encoder(x_wav)
        #x_speaker = x_wav

        x_speaker = torch.flatten(x_wav, start_dim=1)
        
        x_speaker = torch.matmul(F.softmax(self.attention_pool_speaker(x_wav), dim=1).transpose(-1, -2), x_wav) \
            .squeeze(-2)
        
        x_speaker = self.fc_wav(x_speaker)
        """
        """
        x_spect = self.norm_spect(x_spect)
        x_spect = self.fc_spect(x_spect)
        """
        if self.after_fc_embedded:
            x_spect = self.norm_spect(x_spect)
            x_spect = self.fc_spect(x_spect)
            if x_speaker is not None:
                x_speaker = self.project_speaker(x_speaker)
            if x_gender is not None:
                x_gender = self.project_gender(x_gender)
            if x_corpus is not None:
                x_corpus = self.project_corpus(x_corpus)

        else:
            x_speaker = torch.matmul(F.softmax(self.speaker_attention_pool(x_speaker), dim=1).transpose(-1, -2), x_speaker).squeeze(-2)

        if self.union_type == 'concatenate':
            x = torch.cat((x_spect, x_speaker), 1)
        elif self.union_type == 'sum':
            x = torch.sum(torch.stack([x_spect, x_speaker]), dim=0)
        elif self.union_type == 'average':
            x = torch.mean(torch.stack([x_spect, x_speaker]), dim=0)

        elif self.union_type == 'self-attention':

            if x_speaker is not None and x_gender is not None and x_corpus is not None:
                x = torch.stack((x_spect, x_speaker, x_gender, x_corpus), 1)
            elif x_speaker is None:
                if x_gender is not None and x_corpus is not None:
                    x = torch.stack((x_spect, x_gender, x_corpus), 1)
                elif x_gender is None and x_corpus is not None:
                    x = torch.stack((x_spect, x_corpus), 1)
                else:
                    x = torch.stack((x_spect, x_speaker), 1)
            elif x_gender is None:
                if x_speaker is not None and x_corpus is not None:
                    x = torch.stack((x_spect, x_speaker, x_corpus), 1)
                elif x_speaker is None and x_corpus is not None:
                    x = torch.stack((x_spect, x_corpus), 1)
                else:
                    x = torch.stack((x_spect, x_speaker), 1)
            elif x_corpus is None:
                if x_speaker is not None and x_gender is not None:
                    x = torch.stack((x_spect, x_speaker, x_gender), 1)
                elif x_speaker is None and x_gender is not None:
                    x = torch.stack((x_spect, x_gender), 1)
                else:
                    x = torch.stack((x_spect, x_speaker), 1)
            """
            x = self.self_attn(x)[:, 0, :]
            """

            #x += self.positional_emb_scnd_level
            for blk in self.blocks_scnd_level:
                x = blk(x)
            x = x[:, 0, :]

        x = self.norm_union(x)
        """
        #regularization
        sum_batch_sample = 0
        for batch_sample_index in np.arange(0, x.shape[0]):
            sum_batch_sample += x[batch_sample_index].norm(dim=0, p=1)
        
        self.fc.requires_grad = False
        
        regularization_vector = x.shape[0] / (self.num_classes * sum_batch_sample.item())
        for fc_weight_index in np.arange(0, self.fc.weight.shape[0]):
            #fc_column_weights = self.fc.weight[fc_weight_index, :]
            self.fc.weight[fc_weight_index, :] = torch.from_numpy(np.repeat(regularization_vector, self.fc.weight.shape[1]))
        """

        x = self.fc(x)
        return x

    @staticmethod
    def init_weight(m):
        if isinstance(m, Linear):
            init.normal_(m.weight, std=.02)
            if isinstance(m, Linear) and m.bias is not None:
                init.constant_(m.bias, 0)
        elif isinstance(m, LayerNorm):
            init.constant_(m.bias, 0)
            init.constant_(m.weight, 1.0)

    @staticmethod
    def sinusoidal_embedding(n_channels, dim):
        pe = torch.FloatTensor([[p / (10000 ** (2 * (i // 2) / dim)) for i in range(dim)]
                                for p in range(n_channels)])
        pe[:, 0::2] = torch.sin(pe[:, 0::2])
        pe[:, 1::2] = torch.cos(pe[:, 1::2])
        return pe.unsqueeze(0)

def _read_embedder_model(embedder_path, num_embedder_classes=11):
    model_name = 'mcct_7_3x1_32_c100'
    create_fn = model_entrypoint(model_name)
    with set_layer_config(scriptable=None, exportable=None, no_jit=None):
        model = create_fn(num_classes=num_embedder_classes)

    if os.path.splitext(embedder_path)[-1].lower() in ('.npz', '.npy'):
        # numpy checkpoint, try to load via model specific load_pretrained fn
        if hasattr(model, 'load_pretrained'):
            model.load_pretrained(embedder_path)
        else:
            raise NotImplementedError('Model cannot load numpy checkpoint')
        return
    state_dict = load_state_dict(embedder_path, False)
    model.load_state_dict(state_dict, strict=True)
    return model

def _speaker_cct(pretrained, progress, pretrained_arch,
                 num_layers, num_heads, mlp_ratio, embedding_dim, speaker=True, gender=False, corpus=False,
                 speaker_embedder_path=None, gender_embedder_path=None,
                 corpus_embedder_path=None, kernel_size=3, stride=None, padding=None, positional_embedding='learnable',
                 type='self-attention', *args, **kwargs):
    stride = stride if stride is not None else max(1, (kernel_size // 2) - 1)
    padding = padding if padding is not None else max(1, (kernel_size // 2))
    speaker_embedder = None
    if speaker:
        speaker_embedder = _read_embedder_model(speaker_embedder_path)
    gender_embedder = None
    if gender:
        gender_embedder = _read_embedder_model(gender_embedder_path, num_embedder_classes=2)
    corpus_embedder = None
    if corpus:
        corpus_embedder = _read_embedder_model(corpus_embedder_path, num_embedder_classes=5)
    model = SpeakerCCT(speaker_embedder=speaker_embedder,
                       gender_embedder=gender_embedder,
                       corpus_embedder=corpus_embedder,
                       num_layers=num_layers,
                      num_heads=num_heads,
                      mlp_ratio=mlp_ratio,
                      embedding_dim=embedding_dim,
                      kernel_size=kernel_size,
                      stride=stride,
                      padding=padding,
                      type=type,
                      *args, **kwargs)


    if pretrained:
        if pretrained_arch in model_urls:
            state_dict = load_state_dict_from_url(model_urls[pretrained_arch],
                                                  progress=progress)
            if positional_embedding == 'learnable':
                state_dict = pe_check(model, state_dict)
            elif positional_embedding == 'sine':
                state_dict['classifier.positional_emb'] = model.state_dict()['classifier.positional_emb']

            model_dict = model.state_dict()

            state_dict = {k: v for k, v in state_dict.items() if
                                k in model_dict and v.size() == model_dict[k].size()}
            model_dict.update(state_dict)
            model.load_state_dict(state_dict, strict=False)
        else:
            raise RuntimeError(f'Variant {pretrained_arch} does not yet have pretrained weights.')

    return model

def speaker_cct_4(pretrained, progress, type, *args, **kwargs):
    return _speaker_cct( pretrained=pretrained, progress=progress,
                         num_layers=4, num_heads=2, mlp_ratio=1, embedding_dim=128,
                        type=type, *args, **kwargs)

def speaker_cct_7(pretrained, progress, type, *args, **kwargs):
    return _speaker_cct( pretrained=pretrained, progress=progress,
                         num_layers=7, num_heads=4, mlp_ratio=2, embedding_dim=256,
                         type=type, *args, **kwargs)

@register_model
def speaker_cct_7_3x1_32(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                         speaker_embedder_path=None,
                        *args, **kwargs):
    return speaker_cct_7(arch='speaker_cct_7_3x1_32', pretrained=pretrained, progress=progress,  pretrained_arch="cct_7_3x1_32_c100",
                                 kernel_size=3, n_conv_layers=1,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='self-attention',
                                 speaker_embedder_path=speaker_embedder_path,
                                 n_layers_scnd_transformer=1,
                                 *args, **kwargs)

@register_model
def speaker_gender_cct_7_3x1_32(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                         speaker_embedder_path=None,
                         gender_embedder_path=None,
                        *args, **kwargs):
    return speaker_cct_7(arch='speaker_gender_cct_7_3x1_32', pretrained=pretrained, progress=progress,  pretrained_arch="cct_7_3x1_32_c100",
                                 kernel_size=3, n_conv_layers=1,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='self-attention',
                                 speaker_embedder_path=speaker_embedder_path,
                                 gender_embedder_path=gender_embedder_path,
                                 gender=True,
                                 n_layers_scnd_transformer=2,
                                 *args, **kwargs)

@register_model
def speaker_corpus_cct_7_3x1_32(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                         speaker_embedder_path=None,
                         corpus_embedder_path=None,
                        *args, **kwargs):
    return speaker_cct_7(arch='speaker_corpus_cct_7_3x1_32', pretrained=pretrained, progress=progress,  pretrained_arch="cct_7_3x1_32_c100",
                                 kernel_size=3, n_conv_layers=1,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='self-attention',
                                 speaker_embedder_path=speaker_embedder_path,
                                 corpus_embedder_path=corpus_embedder_path,
                                 corpus=True,
                                 n_layers_scnd_transformer=2,
                                 *args, **kwargs)


@register_model
def speaker_gender_corpus_cct_7_3x1_32(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                         speaker_embedder_path = None,
                         gender_embedder_path=None,
                         corpus_embedder_path=None,
                        *args, **kwargs):
    return speaker_cct_7(arch='speaker_gender_corpus_cct_7_3x1_32', pretrained=pretrained, progress=progress,  pretrained_arch="cct_7_3x1_32_c100",
                                 kernel_size=3, n_conv_layers=1,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='self-attention',
                                 speaker_embedder_path=speaker_embedder_path,
                                 gender_embedder_path=gender_embedder_path,
                                 corpus_embedder_path=corpus_embedder_path,
                                    gender=True, corpus=True,
                                    *args, **kwargs)

@register_model
def speaker_cct_7_3x1_32_concatenate(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                         speaker_embedder_path = None,
                        *args, **kwargs):
    return speaker_cct_7(arch='speaker_cct_7_3x1_32', pretrained=pretrained, progress=progress,  pretrained_arch="cct_7_3x1_32_c100",
                                 kernel_size=3, n_conv_layers=1,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='concatenate',
                                 speaker_embedder_path=speaker_embedder_path,
                                 *args, **kwargs)

@register_model
def speaker_cct_7_3x1_32_sum(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                             speaker_embedder_path=None,
                             *args, **kwargs):
    return speaker_cct_7(arch='speaker_cct_7_3x1_32_sum', pretrained=pretrained, progress=progress,  pretrained_arch="cct_7_3x1_32_c100",
                                 kernel_size=3, n_conv_layers=1,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='sum',
                                 speaker_embedder_path=speaker_embedder_path,
                                *args, **kwargs)

@register_model
def speaker_cct_7_3x1_32_average(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                                 speaker_embedder_path=None,
                                 *args, **kwargs):
    return speaker_cct_7(arch='speaker_cct_7_3x1_32_average', pretrained=pretrained, progress=progress,  pretrained_arch="cct_7_3x1_32_c100",
                                 kernel_size=3, n_conv_layers=1,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='average',
                                 speaker_embedder_path=speaker_embedder_path,
                                 *args, **kwargs)

@register_model
def speaker_cct_4_3x1_32(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                         speaker_embedder_path=None,
                         *args, **kwargs):
    return speaker_cct_4(arch='speaker_cct_7_3x1_32', pretrained=pretrained, progress=progress,  pretrained_arch="",
                                 kernel_size=3, n_conv_layers=1,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='concatenate',
                                 speaker_embedder_path=speaker_embedder_path,
                                 *args, **kwargs)

@register_model
def speaker_cct_4_3x1_32_sum(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                             speaker_embedder_path=None,
                             *args, **kwargs):
    return speaker_cct_4(arch='speaker_cct_7_3x1_32_sum', pretrained=pretrained, progress=progress,  pretrained_arch="",
                                 kernel_size=3, n_conv_layers=1,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='sum',
                                 speaker_embedder_path=speaker_embedder_path,
                                 *args, **kwargs)

@register_model
def speaker_cct_4_3x1_32_average(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                                 speaker_embedder_path=None,
                                 *args, **kwargs):
    return speaker_cct_4(arch='speaker_cct_7_3x1_32_average', pretrained=pretrained, progress=progress,  pretrained_arch="",
                                 kernel_size=3, n_conv_layers=1,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes, type='average',
                                 speaker_embedder_path=speaker_embedder_path,
                                 *args, **kwargs)


KeyboardInterrupt: 

# SPEAKER VGGVox CCT

In [ ]:
%%writefile speaker_vgg_vox_cct.py

import os
from tokenizer import Tokenizer
from transformers_component import TransformerEncoderLayer
from helpers import pe_check
import torch.nn as nn
from timm.models.helpers import load_state_dict
from timm.models.layers import set_layer_config

import torch
from torch.nn import Linear, Module, ModuleList, Linear, Dropout, LayerNorm, Parameter, init, LSTM
import torch.nn.functional as F
import librosa
from torch.hub import load_state_dict_from_url
import numpy as np

try:
    from timm.models.registry import register_model, model_entrypoint
except ImportError:
    pass

model_urls = {
    'cct_7_3x1_32':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_cifar10_300epochs.pth',
    'cct_7_3x1_32_sine':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_sine_cifar10_5000epochs.pth',
    'cct_7_3x1_32_c100':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_cifar100_300epochs.pth',
    'cct_7_3x1_32_sine_c100':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_3x1_32_sine_cifar100_5000epochs.pth',
    'cct_7_7x2_224_sine':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_7_7x2_224_flowers102.pth',
    'cct_14_7x2_224':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/pretrained/cct_14_7x2_224_imagenet.pth',
    'cct_14_7x2_384':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/finetuned/cct_14_7x2_384_imagenet.pth',
    'cct_14_7x2_384_fl':
        'http://ix.cs.uoregon.edu/~alih/compact-transformers/checkpoints/finetuned/cct_14_7x2_384_flowers102.pth',
}


class SpeakerVGGVoxCCT(nn.Module):
    def __init__(self,
                 img_size=224,
                 embedding_dim=768,
                 n_input_channels=3,
                 n_conv_layers=1,
                 kernel_size=7,
                 stride=2,
                 padding=3,
                 pooling_kernel_size=3,
                 pooling_stride=2,
                 pooling_padding=1,
                 dropout=0.,
                 attention_dropout=0.1,
                 stochastic_depth=0.1,
                 num_layers=14,
                 num_heads=6,
                 mlp_ratio=4.0,
                 num_classes=1000,
                 positional_embedding='learnable',
                 n_layers_scnd_transformer=4,
                 n_heads_scnd_transformer=2,
                 *args, **kwargs):
        super(SpeakerVGGVoxCCT, self).__init__()

        self.tokenizer = Tokenizer(n_input_channels=n_input_channels,
                                   n_output_channels=embedding_dim,
                                   kernel_size=kernel_size,
                                   stride=stride,
                                   padding=padding,
                                   pooling_kernel_size=pooling_kernel_size,
                                   pooling_stride=pooling_stride,
                                   pooling_padding=pooling_padding,
                                   max_pool=True,
                                   activation=nn.ReLU,
                                   n_conv_layers=n_conv_layers,
                                   conv_bias=False)

        self.classifier = SpeakerVGGVoxCCTClassifier(
            sequence_length=self.tokenizer.sequence_length(n_channels=n_input_channels,
                                                           height=img_size,
                                                           width=img_size),
            embedding_dim=embedding_dim,
            dropout=dropout,
            attention_dropout=attention_dropout,
            stochastic_depth=stochastic_depth,
            num_layers=num_layers,
            num_heads=num_heads,
            mlp_ratio=mlp_ratio,
            num_classes=num_classes,
            positional_embedding=positional_embedding,
            n_layers_scnd_transformer=n_layers_scnd_transformer,
            n_heads_scnd_transformer=n_heads_scnd_transformer,
        )

    def forward(self, x):
        x_emotion = x[0]
        x_speaker = x[1]
        x_emotion = self.tokenizer(x_emotion)
        return self.classifier(x_emotion, x_speaker)


class SpeakerVGGVoxCCTClassifier(Module):
    def __init__(self,
                 embedding_dim=768,
                 num_layers=12,
                 num_heads=12,
                 mlp_ratio=4.0,
                 num_classes=1000,
                 dropout=0.1,
                 attention_dropout=0.1,
                 stochastic_depth=0.1,
                 positional_embedding='learnable',
                 sequence_length=None,
                 n_layers_scnd_transformer=4,
                 n_heads_scnd_transformer=2,
                 ):
        super().__init__()
        positional_embedding = positional_embedding if \
            positional_embedding in ['sine', 'learnable', 'none'] else 'sine'
        dim_feedforward = int(embedding_dim * mlp_ratio)
        self.embedding_dim = embedding_dim
        self.sequence_length = sequence_length
        self.num_tokens = 0
        self.num_classes = num_classes

        assert sequence_length is not None or positional_embedding == 'none', \
            f"Positional embedding is set to {positional_embedding} and" \
            f" the sequence length was not specified."

        self.attention_pool_cct = Linear(self.embedding_dim, 1)
        # self.attention_pool_speaker = Linear(self.embedding_dim, 1)

        if positional_embedding != 'none':
            if positional_embedding == 'learnable':
                self.positional_emb = Parameter(torch.zeros(1, sequence_length, embedding_dim),
                                                requires_grad=True)
                init.normal_(self.positional_emb, std=0.2)
            else:
                self.positional_emb = Parameter(self.sinusoidal_embedding(sequence_length, embedding_dim),
                                                requires_grad=False)
        else:
            self.positional_emb = None

        self.dropout = Dropout(p=dropout)
        dpr = [x.item() for x in torch.linspace(0, stochastic_depth, num_layers)]
        self.blocks = ModuleList([
            TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads,
                                    dim_feedforward=dim_feedforward, dropout=dropout,
                                    attention_dropout=attention_dropout, drop_path_rate=dpr[i])
            for i in range(num_layers)])
        self.norm_cct = LayerNorm(embedding_dim)

        self.lstm = LSTM(embedding_dim, embedding_dim)
        self.norm = LayerNorm(embedding_dim)

        self.blocks_scnd_level = ModuleList([
            TransformerEncoderLayer(d_model=embedding_dim, nhead=n_heads_scnd_transformer,
                                    dim_feedforward=dim_feedforward, dropout=dropout,
                                    attention_dropout=attention_dropout, drop_path_rate=dpr[i])
            for i in range(n_layers_scnd_transformer)])
        self.positional_emb_scnd_level = Parameter(self.sinusoidal_embedding(3, embedding_dim),
                                                   requires_grad=False)
        self.project_vgg_features = Linear(1024, embedding_dim)

        self.norm_after_blocks = LayerNorm(embedding_dim)
        self.fc = Linear(embedding_dim, num_classes)

        self.apply(self.init_weight)

    def forward(self, x_emotion, x_speaker):
        if self.positional_emb is None and x_emotion.size(1) < self.sequence_length:
            x_emotion = F.pad(x_emotion, (0, 0, 0, self.n_channels - x_emotion.size(1)), mode='constant', value=0)

        if self.positional_emb is not None:
            x_emotion += self.positional_emb

        x_emotion = self.dropout(x_emotion)

        for blk in self.blocks:
            x_emotion = blk(x_emotion)
        # x_spect = self.lstm(x_spect)[0]
        x_emotion = self.norm_cct(x_emotion)
        x_emotion = torch.matmul(F.softmax(self.attention_pool_cct(x_emotion), dim=1).transpose(-1, -2),
                                 x_emotion).squeeze(
            -2)

        x_speaker = self.project_vgg_features(x_speaker)

        x = torch.stack((x_emotion, x_speaker), 1)

        # x += self.positional_emb_scnd_level
        for blk in self.blocks_scnd_level:
            x = blk(x)
        x = x[:, 0, :]

        x = self.norm_after_blocks(x)

        x = self.fc(x)
        return x

    @staticmethod
    def init_weight(m):
        if isinstance(m, Linear):
            init.normal_(m.weight, std=.02)
            if isinstance(m, Linear) and m.bias is not None:
                init.constant_(m.bias, 0)
        elif isinstance(m, LayerNorm):
            init.constant_(m.bias, 0)
            init.constant_(m.weight, 1.0)

    @staticmethod
    def sinusoidal_embedding(n_channels, dim):
        pe = torch.FloatTensor([[p / (10000 ** (2 * (i // 2) / dim)) for i in range(dim)]
                                for p in range(n_channels)])
        pe[:, 0::2] = torch.sin(pe[:, 0::2])
        pe[:, 1::2] = torch.cos(pe[:, 1::2])
        return pe.unsqueeze(0)


def _speaker_vgg_cct(pretrained, progress, pretrained_arch,
                     num_layers, num_heads, mlp_ratio, embedding_dim,
                     kernel_size=3, stride=None, padding=None, positional_embedding='learnable',
                     *args, **kwargs):
    stride = stride if stride is not None else max(1, (kernel_size // 2) - 1)
    padding = padding if padding is not None else max(1, (kernel_size // 2))
    model = SpeakerVGGVoxCCT(num_layers=num_layers,
                             num_heads=num_heads,
                             mlp_ratio=mlp_ratio,
                             embedding_dim=embedding_dim,
                             kernel_size=kernel_size,
                             stride=stride,
                             padding=padding,
                             *args, **kwargs)

    if pretrained:
        if pretrained_arch in model_urls:
            state_dict = load_state_dict_from_url(model_urls[pretrained_arch],
                                                  progress=progress)
            if positional_embedding == 'learnable':
                state_dict = pe_check(model, state_dict)
            elif positional_embedding == 'sine':
                state_dict['classifier.positional_emb'] = model.state_dict()['classifier.positional_emb']

            model_dict = model.state_dict()

            state_dict = {k: v for k, v in state_dict.items() if
                          k in model_dict and v.size() == model_dict[k].size()}
            model_dict.update(state_dict)
            model.load_state_dict(state_dict, strict=False)
        else:
            raise RuntimeError(f'Variant {pretrained_arch} does not yet have pretrained weights.')

    return model


def speaker_vgg_cct_7(pretrained, progress, *args, **kwargs):
    return _speaker_vgg_cct(pretrained=pretrained, progress=progress,
                            num_layers=7, num_heads=4, mlp_ratio=2, embedding_dim=256,
                            *args, **kwargs)


@register_model
def speaker_vgg_cct_7_3x1_32(pretrained=False, progress=False, img_size=32, positional_embedding='learnable',
                             num_classes=10,
                             *args, **kwargs):
    return speaker_vgg_cct_7(arch='speaker_vgg_cct_7_3x1_32', pretrained=pretrained, progress=progress,
                             pretrained_arch="cct_7_3x1_32_c100",
                             kernel_size=3, n_conv_layers=1,
                             img_size=img_size, positional_embedding=positional_embedding,
                             num_classes=num_classes,
                             n_layers_scnd_transformer=1,
                             n_heads_scnd_transformer=1,
                             *args, **kwargs)

            model_dict = model.state_dict()

            state_dict = {k: v for k, v in state_dict.items() if
                                k in model_dict and v.size() == model_dict[k].size()}
            model_dict.update(state_dict)
            model.load_state_dict(state_dict, strict=False)
        else:
            raise RuntimeError(f'Variant {pretrained_arch} does not yet have pretrained weights.')

    return model



def speaker_vgg_cct_7(pretrained, progress, *args, **kwargs):
    return _speaker_vgg_cct( pretrained=pretrained, progress=progress,
                         num_layers=7, num_heads=4, mlp_ratio=2, embedding_dim=256,
                         *args, **kwargs)

@register_model
def speaker_vgg_cct_7_3x1_32(pretrained=False, progress=False, img_size=32, positional_embedding='learnable', num_classes=10,
                        *args, **kwargs):
    return speaker_vgg_cct_7(arch='speaker_vgg_cct_7_3x1_32', pretrained=pretrained, progress=progress,  pretrained_arch="cct_7_3x1_32_c100",
                                 kernel_size=3, n_conv_layers=1,
                                 img_size=img_size, positional_embedding=positional_embedding,
                                 num_classes=num_classes,
                                 n_layers_scnd_transformer=1,
                                 n_heads_scnd_transformer=1,
                                 *args, **kwargs)
     

# INIT

In [ ]:
%%writefile __init__.py

from speaker_vgg_vox_cct import *
from cct import *

"""
from .cvt import *
from .vit import *
from .memory_cct import *
from .memory_cvt import *
from .parallel_memory_vit import *
from .speaker_cct import *
from .hybrid_cct import *
"""

# CUSTOM DATASET

In [ ]:
%%writefile parser_csv.py

import pickle

from timm.data.parsers.parser import Parser

class ParserCSV(Parser):
    def __init__(self, csv_dataset_path, class_to_idx, read_wav=False, read_vgg_features=False):
        super().__init__()

        self.samples, self.classes = self.__read_all_dataset_files(csv_dataset_path)

        self.wav_data = []
        if read_wav:
            self.wav_data = self.__read_all_wav_files(csv_dataset_path)
        self.vgg_features = {}
        if read_vgg_features:
            self.vgg_features = self.__read_vgg_vox_features()
        self.class_to_idx = class_to_idx

    def __read_all_dataset_files(self, csv_dataset_path):
        dataset_csv = open(csv_dataset_path)
        csvreader = csv.reader(dataset_csv)
        all_dataset_files = []
        classes_files = []
        for idx, row in enumerate(csvreader):
            if idx > 0:
                all_dataset_files.append(row[0])
                classes_files.append(row[1])
        return all_dataset_files, classes_files

    def __getitem__(self, index):
        path = self.samples[index]
        target = self.class_to_idx[self.classes[index]]
        if len(self.wav_data):
            waveform, _ = torchaudio.load(self.wav_data[index])
            return (open(path, "rb"), self.wav_data[index]), target
        if len(self.vgg_features):
          vgg_features = self.vgg_features[os.path.basename(path).split('.')[0]]
          return (open(path, "rb"), vgg_features), target
        return open(path, "rb"), target

    def __len__(self):
        return len(self.samples)

    def __read_all_wav_files(self, csv_dataset_path):
        dataset_csv = open(csv_dataset_path)
        csvreader = csv.reader(dataset_csv)
        all_wav_files = []
        for idx, row in enumerate(csvreader):
            if idx > 0:
                spectrogram_path = row[0]
                spectrogram_filename = os.path.basename(spectrogram_path)
                original_wav_filename = self.__remove_augmentation_from_filename(spectrogram_filename)
                root_spectrogram_path = os.path.split(os.path.split(spectrogram_path)[0])[0]
                dataset = os.path.basename(root_spectrogram_path)
                wav_folder = '_'.join([dataset, 'wav'])
                wav_path = os.path.join(root_spectrogram_path, wav_folder, original_wav_filename)
                all_wav_files.append(wav_path)
        return all_wav_files
      
    def __read_vgg_vox_features(self):
        first_spectrogram_path =self.samples[0]
        root_path = os.path.split(os.path.split(first_spectrogram_path)[0])[0]
        dataset = os.path.basename(root_path)
        vgg_features_folder = '_'.join([dataset, 'vgg_vox_features'])
        vgg_features_path = os.path.join(root_path, vgg_features_folder, "vggvox_features.pkl")
       
        with open(vgg_features_path, 'rb') as handle:
          vgg_features = pickle.load(handle)
        return vgg_features

    def __remove_augmentation_from_filename(self, filename):
        augmentations_methods = ["loud", "noise", "speed", "pitch", "shift", "norm", "mask", "maskfreq"]
        name, _ = os.path.splitext(filename)
        splitted_filename = name.split("_")

        last_part_filename = splitted_filename[len(splitted_filename)-1]
        if last_part_filename in augmentations_methods:
            '_'.join(name.split('_')[:-1])

        filename_without_last_part = '.'.join([name, 'wav'])

        return filename_without_last_part

In [ ]:
%%writefile speaker_vgg_dataset.py

import librosa
import torch
import torch.utils.data as data
import numpy as np
import logging

import torchaudio
from PIL import Image

from timm.data.parsers import create_parser

from torchaudio.models import Wav2Vec2Model

_logger = logging.getLogger(__name__)

_ERROR_RETRY = 50
class SpeakerVGGDataset(data.Dataset):

    def __init__(
            self,
            root,
            parser=None,
            class_map=None,
            load_bytes=False,
            transform=None,
            target_transform=None,
    ):
        if parser is None or isinstance(parser, str):
            parser = create_parser(parser or '', root=root, class_map=class_map)
        self.parser = parser
        self.load_bytes = load_bytes
        self.transform = transform
        self.target_transform = target_transform
        self._consecutive_errors = 0

    def __getitem__(self, index):
        input, target = self.parser[index]
        img = input[0]
        vgg_features = input[1]
        try:
            img = img.read() if self.load_bytes else Image.open(img).convert('RGB')
        except Exception as e:
            _logger.warning(f'Skipped sample (index {index}, file {self.parser.filename(index)}). {str(e)}')
            self._consecutive_errors += 1
            if self._consecutive_errors < _ERROR_RETRY:
                return self.__getitem__((index + 1) % len(self.parser))
            else:
                raise e
        self._consecutive_errors = 0
        if self.transform is not None:
            img = self.transform(img)
        if target is None:
            target = -1
        elif self.target_transform is not None:
            target = self.target_transform(target)

        return (img, vgg_features), target

    def __len__(self):
        return len(self.parser)

    def filename(self, index, basename=False, absolute=False):
        return self.parser.filename(index, basename, absolute)

    def filenames(self, basename=False, absolute=False):
        return self.parser.filenames(basename, absolute)

In [ ]:
%%writefile custom_loader.py


""" Loader Factory, Fast Collate, CUDA Prefetcher

Prefetcher and Fast Collate inspired by NVIDIA APEX example at
https://github.com/NVIDIA/apex/commit/d5e2bb4bdeedd27b1dfaf5bb2b24d6c000dee9be#diff-cf86c282ff7fba81fad27a559379d5bf

Hacked together by / Copyright 2019, Ross Wightman
"""
import random
from functools import partial
from typing import Callable

import torch.utils.data
import numpy as np

from timm.data.transforms_factory import create_transform
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.data.distributed_sampler import OrderedDistributedSampler, RepeatAugSampler
from timm.data.random_erasing import RandomErasing
from timm.data.mixup import FastCollateMixup


def fast_collate(batch):
    """ A fast collation function optimized for uint8 images (np array or torch) and int64 targets (labels)"""
    assert isinstance(batch[0], tuple)
    batch_size = len(batch)
    if isinstance(batch[0][0], tuple):
        targets = torch.zeros(batch_size, dtype=torch.int64)
        tensor_emotion_spect = torch.zeros((batch_size, *batch[0][0][0].shape), dtype=torch.uint8)
        tensor_speaker_emb = torch.zeros((batch_size, *batch[0][0][1].shape), dtype=torch.float32)

        for i in range(batch_size):
            targets[i] = batch[i][1]
            tensor_emotion_spect[i] += torch.from_numpy(batch[i][0][0])
            tensor_speaker_emb[i] += torch.from_numpy(batch[i][0][1])
        return (tensor_emotion_spect, tensor_speaker_emb), targets

    elif isinstance(batch[0][0], np.ndarray):
        targets = torch.tensor([b[2] for b in batch], dtype=torch.int64)
        assert len(targets) == batch_size
        tensor_img = torch.zeros((batch_size, *batch[0][0].shape), dtype=torch.uint8)
        for i in range(batch_size):
            tensor_img[i] += torch.from_numpy(batch[i][0])
        tensor_wav = torch.zeros((batch_size, *batch[0][1].shape), dtype=torch.float32)
        for i in range(batch_size):
            tensor_wav[i] += torch.from_numpy(batch[i][1])
        return tensor_img, tensor_wav, targets

    elif isinstance(batch[0][0], torch.Tensor):
        targets = torch.tensor([b[1] for b in batch], dtype=torch.int64)
        assert len(targets) == batch_size
        tensor = torch.zeros((batch_size, *batch[0][0].shape), dtype=torch.uint8)
        for i in range(batch_size):
            tensor[i].copy_(batch[i][0])
        return tensor, targets
    else:
        assert False


class PrefetchLoader:

    def __init__(self,
                 loader,
                 mean=IMAGENET_DEFAULT_MEAN,
                 std=IMAGENET_DEFAULT_STD,
                 fp16=False,
                 re_prob=0.,
                 re_mode='const',
                 re_count=1,
                 re_num_splits=0):
        self.loader = loader
        self.mean = torch.tensor([x * 255 for x in mean]).cuda().view(1, 3, 1, 1)
        self.std = torch.tensor([x * 255 for x in std]).cuda().view(1, 3, 1, 1)
        self.fp16 = fp16
        if fp16:
            self.mean = self.mean.half()
            self.std = self.std.half()

        if re_prob > 0.:
            self.random_erasing = RandomErasing(
                probability=re_prob, mode=re_mode, max_count=re_count, num_splits=re_num_splits)
        else:
            self.random_erasing = None


    def __iter__(self):
        stream = torch.cuda.Stream()
        first = True
        for next_input, next_target in self.loader:
            next_spect = next_input[0]
            next_speaker_emb = next_input[1]
            #print(next_spect.shape)
            #print(next_speaker_emb.shape)
            #print(next_target.shape)
            with torch.cuda.stream(stream):
                next_spect = next_spect.cuda(non_blocking=True)
                next_speaker_emb = next_speaker_emb.cuda(non_blocking=True)
                next_target = next_target.cuda(non_blocking=True)
                if self.fp16:
                    next_spect = next_spect.half().sub_(self.mean).div_(self.std)
                else:
                    next_spect = next_spect.float().sub_(self.mean).div_(self.std)
                if self.random_erasing is not None:
                    next_spect = self.random_erasing(next_spect)

            if not first:
                yield (input_spect, input_speaker_emb), target
            else:
                first = False

            torch.cuda.current_stream().wait_stream(stream)
            input_spect = next_spect
            input_speaker_emb = next_speaker_emb
            target = next_target

        yield (input_spect, input_speaker_emb), target

    def __len__(self):
        return len(self.loader)

    @property
    def sampler(self):
        return self.loader.sampler

    @property
    def dataset(self):
        return self.loader.dataset

    @property
    def mixup_enabled(self):
        if isinstance(self.loader.collate_fn, FastCollateMixup):
            return self.loader.collate_fn.mixup_enabled
        else:
            return False

    @mixup_enabled.setter
    def mixup_enabled(self, x):
        if isinstance(self.loader.collate_fn, FastCollateMixup):
            self.loader.collate_fn.mixup_enabled = x


def _worker_init(worker_id, worker_seeding='all'):
    worker_info = torch.utils.data.get_worker_info()
    assert worker_info.id == worker_id
    if isinstance(worker_seeding, Callable):
        seed = worker_seeding(worker_info)
        random.seed(seed)
        torch.manual_seed(seed)
        np.random.seed(seed % (2 ** 32 - 1))
    else:
        assert worker_seeding in ('all', 'part')
        # random / torch seed already called in dataloader iter class w/ worker_info.seed
        # to reproduce some old results (same seed + hparam combo), partial seeding is required (skip numpy re-seed)
        if worker_seeding == 'all':
            np.random.seed(worker_info.seed % (2 ** 32 - 1))


def create_custom_loader(
        dataset,
        input_size,
        batch_size,
        is_training=False,
        use_prefetcher=True,
        no_aug=False,
        re_prob=0.,
        re_mode='const',
        re_count=1,
        re_split=False,
        scale=None,
        ratio=None,
        hflip=0.5,
        vflip=0.,
        color_jitter=0.4,
        auto_augment=None,
        num_aug_repeats=0,
        num_aug_splits=0,
        interpolation='bilinear',
        mean=IMAGENET_DEFAULT_MEAN,
        std=IMAGENET_DEFAULT_STD,
        num_workers=1,
        distributed=False,
        crop_pct=None,
        collate_fn=None,
        pin_memory=False,
        fp16=False,
        tf_preprocessing=False,
        use_multi_epochs_loader=False,
        persistent_workers=True,
        worker_seeding='all',
):
    re_num_splits = 0
    if re_split:
        # apply RE to second half of batch if no aug split otherwise line up with aug split
        re_num_splits = num_aug_splits or 2
    dataset.transform = create_transform(
        input_size,
        is_training=is_training,
        use_prefetcher=use_prefetcher,
        no_aug=no_aug,
        scale=scale,
        ratio=ratio,
        hflip=hflip,
        vflip=vflip,
        color_jitter=color_jitter,
        auto_augment=auto_augment,
        interpolation=interpolation,
        mean=mean,
        std=std,
        crop_pct=crop_pct,
        tf_preprocessing=tf_preprocessing,
        re_prob=re_prob,
        re_mode=re_mode,
        re_count=re_count,
        re_num_splits=re_num_splits,
        separate=num_aug_splits > 0,
    )

    sampler = None
    if distributed and not isinstance(dataset, torch.utils.data.IterableDataset):
        if is_training:
            if num_aug_repeats:
                sampler = RepeatAugSampler(dataset, num_repeats=num_aug_repeats)
            else:
                sampler = torch.utils.data.distributed.DistributedSampler(dataset)
        else:
            # This will add extra duplicate entries to result in equal num
            # of samples per-process, will slightly alter validation results
            sampler = OrderedDistributedSampler(dataset)
    else:
        assert num_aug_repeats == 0, "RepeatAugment not currently supported in non-distributed or IterableDataset use"

    if collate_fn is None:
        collate_fn = fast_collate if use_prefetcher else torch.utils.data.dataloader.default_collate

    loader_class = torch.utils.data.DataLoader
    if use_multi_epochs_loader:
        loader_class = MultiEpochsDataLoader

    loader_args = dict(
        batch_size=batch_size,
        shuffle=not isinstance(dataset, torch.utils.data.IterableDataset) and sampler is None and is_training,
        num_workers=num_workers,
        sampler=sampler,
        collate_fn=collate_fn,
        pin_memory=pin_memory,
        drop_last=is_training,
        worker_init_fn=partial(_worker_init, worker_seeding=worker_seeding),
        persistent_workers=persistent_workers
    )
    try:
        loader = loader_class(dataset, **loader_args)
    except TypeError as e:
        loader_args.pop('persistent_workers')  # only in Pytorch 1.7+
        loader = loader_class(dataset, **loader_args)
    if use_prefetcher:
        prefetch_re_prob = re_prob if is_training and not no_aug else 0.
        loader = PrefetchLoader(
            loader,
            mean=mean,
            std=std,
            fp16=fp16,
            re_prob=prefetch_re_prob,
            re_mode=re_mode,
            re_count=re_count,
            re_num_splits=re_num_splits
        )

    return loader


class MultiEpochsDataLoader(torch.utils.data.DataLoader):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._DataLoader__initialized = False
        self.batch_sampler = _RepeatSampler(self.batch_sampler)
        self._DataLoader__initialized = True
        self.iterator = super().__iter__()

    def __len__(self):
        return len(self.batch_sampler.sampler)

    def __iter__(self):
        for i in range(len(self)):
            yield next(self.iterator)


class _RepeatSampler(object):
    """ Sampler that repeats forever.

    Args:
        sampler (Sampler)
    """

    def __init__(self, sampler):
        self.sampler = sampler

    def __iter__(self):
        while True:
            yield from iter(self.sampler)


In [ ]:
%%writefile mixup.py

""" Mixup and Cutmix

Papers:
mixup: Beyond Empirical Risk Minimization (https://arxiv.org/abs/1710.09412)

CutMix: Regularization Strategy to Train Strong Classifiers with Localizable Features (https://arxiv.org/abs/1905.04899)

Code Reference:
CutMix: https://github.com/clovaai/CutMix-PyTorch

Hacked together by / Copyright 2019, Ross Wightman
"""
import numpy as np
import torch


def one_hot(x, num_classes, on_value=1., off_value=0., device='cuda'):
    x = x.long().view(-1, 1)
    return torch.full((x.size()[0], num_classes), off_value, device=device).scatter_(1, x, on_value)


def mixup_target(target, num_classes, lam=1., smoothing=0.0, device='cuda'):
    off_value = smoothing / num_classes
    on_value = 1. - smoothing + off_value
    y1 = one_hot(target, num_classes, on_value=on_value, off_value=off_value, device=device)
    y2 = one_hot(target.flip(0), num_classes, on_value=on_value, off_value=off_value, device=device)
    return y1 * lam + y2 * (1. - lam)

def mixup_vector(vectors, lam=1.):
    y1 = vectors
    y2 = vectors.flip(0)
    return y1 * lam + y2 * (1. - lam)

def rand_bbox(img_shape, lam, margin=0., count=None):
    """ Standard CutMix bounding-box
    Generates a random square bbox based on lambda value. This impl includes
    support for enforcing a border margin as percent of bbox dimensions.

    Args:
        img_shape (tuple): Image shape as tuple
        lam (float): Cutmix lambda value
        margin (float): Percentage of bbox dimension to enforce as margin (reduce amount of box outside image)
        count (int): Number of bbox to generate
    """
    ratio = np.sqrt(1 - lam)
    img_h, img_w = img_shape[-2:]
    cut_h, cut_w = int(img_h * ratio), int(img_w * ratio)
    margin_y, margin_x = int(margin * cut_h), int(margin * cut_w)
    cy = np.random.randint(0 + margin_y, img_h - margin_y, size=count)
    cx = np.random.randint(0 + margin_x, img_w - margin_x, size=count)
    yl = np.clip(cy - cut_h // 2, 0, img_h)
    yh = np.clip(cy + cut_h // 2, 0, img_h)
    xl = np.clip(cx - cut_w // 2, 0, img_w)
    xh = np.clip(cx + cut_w // 2, 0, img_w)
    return yl, yh, xl, xh


def rand_bbox_minmax(img_shape, minmax, count=None):
    """ Min-Max CutMix bounding-box
    Inspired by Darknet cutmix impl, generates a random rectangular bbox
    based on min/max percent values applied to each dimension of the input image.

    Typical defaults for minmax are usually in the  .2-.3 for min and .8-.9 range for max.

    Args:
        img_shape (tuple): Image shape as tuple
        minmax (tuple or list): Min and max bbox ratios (as percent of image size)
        count (int): Number of bbox to generate
    """
    assert len(minmax) == 2
    img_h, img_w = img_shape[-2:]
    cut_h = np.random.randint(int(img_h * minmax[0]), int(img_h * minmax[1]), size=count)
    cut_w = np.random.randint(int(img_w * minmax[0]), int(img_w * minmax[1]), size=count)
    yl = np.random.randint(0, img_h - cut_h, size=count)
    xl = np.random.randint(0, img_w - cut_w, size=count)
    yu = yl + cut_h
    xu = xl + cut_w
    return yl, yu, xl, xu


def cutmix_bbox_and_lam(img_shape, lam, ratio_minmax=None, correct_lam=True, count=None):
    """ Generate bbox and apply lambda correction.
    """
    if ratio_minmax is not None:
        yl, yu, xl, xu = rand_bbox_minmax(img_shape, ratio_minmax, count=count)
    else:
        yl, yu, xl, xu = rand_bbox(img_shape, lam, count=count)
    if correct_lam or ratio_minmax is not None:
        bbox_area = (yu - yl) * (xu - xl)
        lam = 1. - bbox_area / float(img_shape[-2] * img_shape[-1])
    return (yl, yu, xl, xu), lam


class Mixup:
    """ Mixup/Cutmix that applies different params to each element or whole batch

    Args:
        mixup_alpha (float): mixup alpha value, mixup is active if > 0.
        cutmix_alpha (float): cutmix alpha value, cutmix is active if > 0.
        cutmix_minmax (List[float]): cutmix min/max image ratio, cutmix is active and uses this vs alpha if not None.
        prob (float): probability of applying mixup or cutmix per batch or element
        switch_prob (float): probability of switching to cutmix instead of mixup when both are active
        mode (str): how to apply mixup/cutmix params (per 'batch', 'pair' (pair of elements), 'elem' (element)
        correct_lam (bool): apply lambda correction when cutmix bbox clipped by image borders
        label_smoothing (float): apply label smoothing to the mixed target tensor
        num_classes (int): number of classes for target
    """
    def __init__(self, mixup_alpha=1., cutmix_alpha=0., cutmix_minmax=None, prob=1.0, switch_prob=0.5,
                 mode='batch', correct_lam=True, label_smoothing=0.1, num_classes=1000):
        self.mixup_alpha = mixup_alpha
        self.cutmix_alpha = cutmix_alpha
        self.cutmix_minmax = cutmix_minmax
        if self.cutmix_minmax is not None:
            assert len(self.cutmix_minmax) == 2
            # force cutmix alpha == 1.0 when minmax active to keep logic simple & safe
            self.cutmix_alpha = 1.0
        self.mix_prob = prob
        self.switch_prob = switch_prob
        self.label_smoothing = label_smoothing
        self.num_classes = num_classes
        self.mode = mode
        self.correct_lam = correct_lam  # correct lambda based on clipped area for cutmix
        self.mixup_enabled = True  # set to false to disable mixing (intended tp be set by train loop)

    def _params_per_elem(self, batch_size):
        lam = np.ones(batch_size, dtype=np.float32)
        use_cutmix = np.zeros(batch_size, dtype=np.bool)
        if self.mixup_enabled:
            if self.mixup_alpha > 0. and self.cutmix_alpha > 0.:
                use_cutmix = np.random.rand(batch_size) < self.switch_prob
                lam_mix = np.where(
                    use_cutmix,
                    np.random.beta(self.cutmix_alpha, self.cutmix_alpha, size=batch_size),
                    np.random.beta(self.mixup_alpha, self.mixup_alpha, size=batch_size))
            elif self.mixup_alpha > 0.:
                lam_mix = np.random.beta(self.mixup_alpha, self.mixup_alpha, size=batch_size)
            elif self.cutmix_alpha > 0.:
                use_cutmix = np.ones(batch_size, dtype=np.bool)
                lam_mix = np.random.beta(self.cutmix_alpha, self.cutmix_alpha, size=batch_size)
            else:
                assert False, "One of mixup_alpha > 0., cutmix_alpha > 0., cutmix_minmax not None should be true."
            lam = np.where(np.random.rand(batch_size) < self.mix_prob, lam_mix.astype(np.float32), lam)
        return lam, use_cutmix

    def _params_per_batch(self):
        lam = 1.
        use_cutmix = False
        if self.mixup_enabled and np.random.rand() < self.mix_prob:
            if self.mixup_alpha > 0. and self.cutmix_alpha > 0.:
                use_cutmix = np.random.rand() < self.switch_prob
                lam_mix = np.random.beta(self.cutmix_alpha, self.cutmix_alpha) if use_cutmix else \
                    np.random.beta(self.mixup_alpha, self.mixup_alpha)
            elif self.mixup_alpha > 0.:
                lam_mix = np.random.beta(self.mixup_alpha, self.mixup_alpha)
            elif self.cutmix_alpha > 0.:
                use_cutmix = True
                lam_mix = np.random.beta(self.cutmix_alpha, self.cutmix_alpha)
            else:
                assert False, "One of mixup_alpha > 0., cutmix_alpha > 0., cutmix_minmax not None should be true."
            lam = float(lam_mix)
        return lam, use_cutmix

    def _mix_elem(self, x):
        batch_size = len(x)
        lam_batch, use_cutmix = self._params_per_elem(batch_size)
        x_orig = x.clone()  # need to keep an unmodified original for mixing source
        for i in range(batch_size):
            j = batch_size - i - 1
            lam = lam_batch[i]
            if lam != 1.:
                if use_cutmix[i]:
                    (yl, yh, xl, xh), lam = cutmix_bbox_and_lam(
                        x[i].shape, lam, ratio_minmax=self.cutmix_minmax, correct_lam=self.correct_lam)
                    x[i][:, yl:yh, xl:xh] = x_orig[j][:, yl:yh, xl:xh]
                    lam_batch[i] = lam
                else:
                    x[i] = x[i] * lam + x_orig[j] * (1 - lam)
        return torch.tensor(lam_batch, device=x.device, dtype=x.dtype).unsqueeze(1)

    def _mix_pair(self, x):
        batch_size = len(x)
        lam_batch, use_cutmix = self._params_per_elem(batch_size // 2)
        x_orig = x.clone()  # need to keep an unmodified original for mixing source
        for i in range(batch_size // 2):
            j = batch_size - i - 1
            lam = lam_batch[i]
            if lam != 1.:
                if use_cutmix[i]:
                    (yl, yh, xl, xh), lam = cutmix_bbox_and_lam(
                        x[i].shape, lam, ratio_minmax=self.cutmix_minmax, correct_lam=self.correct_lam)
                    x[i][:, yl:yh, xl:xh] = x_orig[j][:, yl:yh, xl:xh]
                    x[j][:, yl:yh, xl:xh] = x_orig[i][:, yl:yh, xl:xh]
                    lam_batch[i] = lam
                else:
                    x[i] = x[i] * lam + x_orig[j] * (1 - lam)
                    x[j] = x[j] * lam + x_orig[i] * (1 - lam)
        lam_batch = np.concatenate((lam_batch, lam_batch[::-1]))
        return torch.tensor(lam_batch, device=x.device, dtype=x.dtype).unsqueeze(1)

    def _mix_batch(self, x):
        lam, use_cutmix = self._params_per_batch()
        if lam == 1.:
            return 1.
        if use_cutmix:
            (yl, yh, xl, xh), lam = cutmix_bbox_and_lam(
                x.shape, lam, ratio_minmax=self.cutmix_minmax, correct_lam=self.correct_lam)
            x[:, :, yl:yh, xl:xh] = x.flip(0)[:, :, yl:yh, xl:xh]
        else:
            x_flipped = x.flip(0).mul_(1. - lam)
            x.mul_(lam).add_(x_flipped)
        return lam

    def __call__(self, x, target):
        assert len(x) % 2 == 0, 'Batch size should be even when using this'
        if self.mode == 'elem':
            lam = self._mix_elem(x)
        elif self.mode == 'pair':
            lam = self._mix_pair(x)
        else:
            lam = self._mix_batch(x)
        target = mixup_target(target, self.num_classes, lam, self.label_smoothing, x.device)
        return x, target


class FastCollateMixup(Mixup):
    """ Fast Collate w/ Mixup/Cutmix that applies different params to each element or whole batch

    A Mixup impl that's performed while collating the batches.
    """

    def _mix_elem_collate(self, output, batch, half=False):
        batch_size = len(batch)
        num_elem = batch_size // 2 if half else batch_size
        assert len(output) == num_elem
        lam_batch, use_cutmix = self._params_per_elem(num_elem)
        for i in range(num_elem):
            j = batch_size - i - 1
            lam = lam_batch[i]
            if isinstance(batch[i][0], tuple):
                mixed = batch[i][0][0]
            else:
                mixed = batch[i][0]
            if lam != 1.:
                if use_cutmix[i]:
                    if not half:
                        mixed = mixed.copy()
                    (yl, yh, xl, xh), lam = cutmix_bbox_and_lam(
                        output.shape, lam, ratio_minmax=self.cutmix_minmax, correct_lam=self.correct_lam)
                    if isinstance(batch[i][0], tuple):
                        mixed[:, yl:yh, xl:xh] = batch[j][0][0][:, yl:yh, xl:xh]
                    else:
                        mixed[:, yl:yh, xl:xh] = batch[j][0][:, yl:yh, xl:xh]
                    lam_batch[i] = lam
                else:
                    if isinstance(batch[i][0], tuple):
                        mixed = mixed.astype(np.float32) * lam + batch[j][0][0].astype(np.float32) * (1 - lam)
                    else:
                        mixed = mixed.astype(np.float32) * lam + batch[j][0].astype(np.float32) * (1 - lam)
                    np.rint(mixed, out=mixed)
            output[i] += torch.from_numpy(mixed.astype(np.uint8))
        if half:
            lam_batch = np.concatenate((lam_batch, np.ones(num_elem)))
        return torch.tensor(lam_batch).unsqueeze(1)

    def _mix_pair_collate(self, output, batch):
        batch_size = len(batch)
        lam_batch, use_cutmix = self._params_per_elem(batch_size // 2)
        for i in range(batch_size // 2):
            j = batch_size - i - 1
            lam = lam_batch[i]
            if isinstance(batch[i][0], tuple):
                mixed_i = batch[i][0][0]
                mixed_j = batch[j][0][0]
            else:
                mixed_i = batch[i][0]
                mixed_j = batch[j][0]
            assert 0 <= lam <= 1.0
            if lam < 1.:
                if use_cutmix[i]:
                    (yl, yh, xl, xh), lam = cutmix_bbox_and_lam(
                        output.shape, lam, ratio_minmax=self.cutmix_minmax, correct_lam=self.correct_lam)
                    patch_i = mixed_i[:, yl:yh, xl:xh].copy()
                    mixed_i[:, yl:yh, xl:xh] = mixed_j[:, yl:yh, xl:xh]
                    mixed_j[:, yl:yh, xl:xh] = patch_i
                    lam_batch[i] = lam
                else:
                    mixed_temp = mixed_i.astype(np.float32) * lam + mixed_j.astype(np.float32) * (1 - lam)
                    mixed_j = mixed_j.astype(np.float32) * lam + mixed_i.astype(np.float32) * (1 - lam)
                    mixed_i = mixed_temp
                    np.rint(mixed_j, out=mixed_j)
                    np.rint(mixed_i, out=mixed_i)
            output[i] += torch.from_numpy(mixed_i.astype(np.uint8))
            output[j] += torch.from_numpy(mixed_j.astype(np.uint8))
        lam_batch = np.concatenate((lam_batch, lam_batch[::-1]))
        return torch.tensor(lam_batch).unsqueeze(1)

    def _mix_batch_collate(self, output, batch):
        batch_size = len(batch)
        lam, use_cutmix = self._params_per_batch()
        if use_cutmix:
            (yl, yh, xl, xh), lam = cutmix_bbox_and_lam(
                output.shape, lam, ratio_minmax=self.cutmix_minmax, correct_lam=self.correct_lam)
        for i in range(batch_size):
            j = batch_size - i - 1
            if isinstance(batch[i][0], tuple):
                mixed = batch[i][0][0]
            else:
                mixed = batch[i][0]
            if lam != 1.:
                if use_cutmix:
                    mixed = mixed.copy()  # don't want to modify the original while iterating
                    if isinstance(batch[i][0], tuple):
                        mixed[:, yl:yh, xl:xh] = batch[j][0][0][:, yl:yh, xl:xh]
                    else:
                        mixed[:, yl:yh, xl:xh] = batch[j][0][:, yl:yh, xl:xh]
                else:
                    if isinstance(batch[i][0], tuple):
                        mixed = mixed.astype(np.float32) * lam + batch[j][0][0].astype(np.float32) * (1 - lam)
                    else:
                        mixed = mixed.astype(np.float32) * lam + batch[j][0].astype(np.float32) * (1 - lam)
                    np.rint(mixed, out=mixed)
            output[i] += torch.from_numpy(mixed.astype(np.uint8))
        return lam

    def __call__(self, batch, _=None):
        batch_size = len(batch)
        assert batch_size % 2 == 0, 'Batch size should be even when using this'
        half = 'half' in self.mode
        if half:
            batch_size //= 2

        if isinstance(batch[0][0], tuple):
            image_shape = batch[0][0][0].shape
        else:
            image_shape = batch[0][0].shape

        output = torch.zeros((batch_size, *image_shape), dtype=torch.uint8)
        if self.mode == 'elem' or self.mode == 'half':
            lam = self._mix_elem_collate(output, batch, half=half)
        elif self.mode == 'pair':
            lam = self._mix_pair_collate(output, batch)
        else:
            lam = self._mix_batch_collate(output, batch)
        target = torch.tensor([b[1] for b in batch], dtype=torch.int64)
        target = mixup_target(target, self.num_classes, lam, self.label_smoothing, device='cpu')
        target = target[:batch_size]
        if isinstance(batch[0][0], tuple):
            other_features = torch.FloatTensor(np.array([b[0][1] for b in batch]))
            other_features = mixup_vector(other_features, lam)
            return (output, other_features), target
        else:
            return output, target

# UTILS

In [ ]:
%%writefile utils.py

import os

import yaml
from matplotlib import pyplot as plt
import numpy as np
import csv
import pandas as pd
import seaborn as sn


def update_graph(train_loss_history, val_loss_history, val_acc_history, path):
    """
    Method that update graphs with the trend of loss and accuracy on the train and validation data
    :param train_loss_history, val_loss_history: list that contains loss value for train and validation detected
                                                 at each epoch
           train_acc_history, val_loss_hystory: list that contains mean accuracy for train and validation
                                                detected at each epoch
           path: path of the directory where to save the graphs
    :return:
    """
    if not os.path.exists(path):
        os.mkdir(path)

    losses_img_file = os.path.join(path, "training_losses.png")
    acc_img_file = os.path.join(path, "training_accuracy.png")
    epochs = np.arange(1, len(train_loss_history) + 1)
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.title("Plot Training/Validation Losses")
    plt.ylim(0, max(max(train_loss_history), max(val_loss_history)))
    plt.plot(epochs, train_loss_history, label="average train loss")
    plt.plot(epochs, val_loss_history, label="average validation loss")
    plt.legend()
    plt.savefig(losses_img_file)
    plt.close()
    plt.title("Plot Validation Accuracy")
    plt.xlabel("epoch")
    plt.ylabel("accuracy(%)")
    plt.ylim(0, 100)
    # plt.plot(epochs, train_acc_history, label="average train accuracy")
    plt.plot(epochs, val_acc_history, label="average validation accuracy")
    plt.legend()
    plt.savefig(acc_img_file)
    plt.close()


def read_history_from_csv(path):
    csv_path = os.path.join(path, "summary.csv")
    train_loss_history, val_loss_history, val_acc_history = [], [], []
    if not os.path.exists(csv_path):
        print("ERROR: not find csv in path ", csv_path)
        return
    with open(csv_path, mode='r') as file:
        for idx, line in enumerate(csv.reader(file)):
            if line[0] != "epoch":
                train_loss_history.append(float(line[1]))
                val_loss_history.append(float(line[2]))
                val_acc_history.append(float(line[3]))
    return train_loss_history, val_loss_history, val_acc_history


def save_confusion_matrix(cm, labels, fname):
    df_cm = pd.DataFrame(cm, index=[str(i) for i in labels],
                         columns=[str(i) for i in labels])
    plt.figure(figsize=(10, 7))
    sn.heatmap(df_cm, annot=True, cmap="Blues")
    plt.savefig(fname, dpi=240)
    plt.close()


def save_performance(performance, path):
    with open(path, 'w') as outfile:
        yaml.dump(performance, outfile, default_flow_style=False)


def get_emotion(filename):
    head, tail = os.path.split(filename)
    tail_split = tail.split("_")
    return tail_split[0]


def get_class_to_idx(dataset, num_classes):
    class_to_idx = {}
    if num_classes == "3_classes":
        class_to_idx = {'neutral': 0, 'positive': 1, 'negative': 2}
    elif num_classes == "4_classes" or num_classes == "4_classes_excited":
        class_to_idx = {'anger': 0, 'happy': 1, 'neutral': 2, "sad": 3}
    elif num_classes == "all_classes":
        if dataset == "EMODB":
            class_to_idx = {'happy': 0, 'sad': 1, 'anger': 2,
                            'disgust': 3, 'fearful': 4, 'neutral': 5}
        elif dataset == "DEMOS":
            class_to_idx = {'happy': 0, 'sad': 1, 'anger': 2,
                            'disgust': 3, 'fearful': 4, 'surprised': 5}
        elif dataset == "IEMOCAP":
            class_to_idx = {'happy': 0, 'sad': 1, 'anger': 2,
                            'disgust': 3, 'fearful': 4, 'surprised': 6, 'neutral': 5,
                            'excited': 7, 'frustration': 8}
        else:
            class_to_idx = {'happy': 0, 'sad': 1, 'anger': 2,
                            'disgust': 3, 'fearful': 4, 'neutral': 5, 'surprised': 6}
    return class_to_idx


def get_3_class_to_idx():
    class_to_idx = {'neutral': 0, 'positive': 1, 'negative': 2}
    return class_to_idx

def get_speaker_class_to_idx():
    class_to_idx = {'demos-01': 0, 'demos-02': 1, 'demos-03': 2, 'demos-04': 3,
                    'demos-05': 4, 'emovo-01': 5, 'emovo-02': 6, 'emovo-03': 7, 'emovo-04': 8,
                    'emovo-05': 9, 'emovo-06': 10}
    return class_to_idx

def get_gender_class_to_idx():
    class_to_idx = {'F': 0, 'M': 1}
    return class_to_idx

def get_corpus_class_to_idx():
    class_to_idx = {"emovo": 0, "iemocap": 1, "ravdess": 2, "savee": 3, "emodb": 4}
    return class_to_idx

def get_class_labels(train_dataset_path):
    classes_label = []
    for root, subdirs, files in os.walk(train_dataset_path):
        for file in files:
            if os.path.splitext(file)[-1].lower() == ".csv":
                dataset_csv = open(os.path.join(root, file))
                csvreader = csv.reader(dataset_csv)
                for idx, row in enumerate(csvreader):
                    if idx > 0:
                        current_class = row[1]
                        if current_class not in classes_label:
                            classes_label.append(current_class)
    return classes_label


def check_read_wav_from_model_name(model_name):
    first_part_name = model_name.split('_')[0]
    if first_part_name == 'speaker':
        return True
    return False


def get_dataset_name_from_path(path):
    all_datasets = ["EMOVO", "IEMOCAP", "RAVDESS", "SAVEE", "EMODB", "DEMOS", "TESS"]
    path_tails = path.split('/')
    for dataset in all_datasets:
        if dataset in path_tails:
            return dataset
    return None

def get_classes_name_from_path(path):
    all_classes_name = ["3_classes", "3_classes_excited", "4_classes", "4_classes_excited", "all_classes",
                        "all_classes_excited"]
    path_tails = path.split('/')
    for classes_name in all_classes_name:
        if classes_name in path_tails:
            return classes_name
    return None

def get_aug_type_from_path(path):
    all_aug = ["no_aug", "balanced_aug", "time_aug", "freq_aug", "time_freq_aud"]
    path_tails = path.split('/')
    for aug in all_aug:
        if aug in path_tails:
            return aug
    return None

def get_exp_type_from_path(path):
    all_exp = ["within-corpus", "IEMOCAP_train", "cross-corpus"]
    path_tails = path.split('/')
    for exp in all_exp:
        if exp in path_tails:
            return exp
    return None



# TRAINING WITHIN CORPUS

**TRAIN**

In [50]:
%%writefile train.py
#!/usr/bin/env python3
# This is a slightly modified version of timm's training script
""" ImageNet Training Script

This is intended to be a lean and easily modifiable ImageNet training script that reproduces ImageNet
training results with some of the latest networks and training techniques. It favours canonical PyTorch
and standard Python style over trying to be able to 'do it all.' That said, it offers quite a few speed
and training result improvements over the usual PyTorch example scripts. Repurpose as you see fit.

This script was started from an early version of the PyTorch ImageNet example
(https://github.com/pytorch/examples/tree/master/imagenet)

NVIDIA CUDA specific speedups adopted from NVIDIA Apex examples
(https://github.com/NVIDIA/apex/tree/master/examples/imagenet)

Hacked together by / Copyright 2020 Ross Wightman (https://github.com/rwightman)
"""
import argparse
import time
import yaml
import os
import logging
from collections import OrderedDict
from contextlib import suppress

from torch.utils.data import DataLoader

"""
from hybrid_dataset import HybridDataset
from speaker_dataset import SpeakerDataset
from utils import update_graph, read_history_from_csv
"""

import torch
import torch.nn as nn
import torchvision.utils
from torch.nn.parallel import DistributedDataParallel as NativeDDP

from timm.data import create_loader, resolve_data_config, Mixup, FastCollateMixup, AugMixDataset, \
    ImageDataset

from timm.models import create_model, safe_model_name, resume_checkpoint, load_checkpoint, \
    convert_splitbn_model, model_parameters
from timm.utils import *
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy, JsdCrossEntropy
from timm.optim import create_optimizer_v2, optimizer_kwargs
from timm.scheduler import create_scheduler
from timm.utils import ApexScaler, NativeScaler
from utils import *
from src import *
from speaker_vgg_dataset import SpeakerVGGDataset
from parser_csv import ParserCSV
from custom_loader import create_custom_loader
from checkpoint import CheckpointSaver as Saver
from mixup import FastCollateMixup as CustomFastCollateMixup

try:

    from apex import amp
    from apex.parallel import DistributedDataParallel as ApexDDP
    from apex.parallel import convert_syncbn_model

    has_apex = True
except ImportError:
    has_apex = False

has_native_amp = False
try:
    if getattr(torch.cuda.amp, 'autocast') is not None:
        has_native_amp = True
except AttributeError:
    pass

try:
    import wandb

    has_wandb = True
except ImportError:
    has_wandb = False
torch.backends.cudnn.benchmark = True
_logger = logging.getLogger('train')

# The first arg parser parses out only the --config argument, this argument is used to
# load a yaml file containing key-values that override the defaults for the main parser below
config_parser = parser = argparse.ArgumentParser(description='Training Config', add_help=False)
parser.add_argument('-c', '--config', default='', type=str, metavar='FILE',
                    help='YAML config file specifying default arguments')

parser = argparse.ArgumentParser(description='PyTorch ImageNet Training')

# Dataset / Model parameters
parser.add_argument('data_train_dir', metavar='DIR',
                    help='path to dataset')
parser.add_argument('--data_eval_dir', metavar='DIR',
                    help='path to validation')
parser.add_argument('--root_output_dir', default="./output/train", metavar='DIR',
                    help='root output dir')

parser.add_argument('--save_graphs', action='store_true', default=False,
                    help='define if save graph of training/validation losses and accuracies')

parser.add_argument('--dataset', '-d', metavar='NAME', default='',
                    help='dataset type (default: ImageFolder/ImageTar if empty)')
parser.add_argument('--train-split', metavar='NAME', default='train',
                    help='dataset train split (default: train)')
parser.add_argument('--val-split', metavar='NAME', default='validation',
                    help='dataset validation split (default: validation)')
parser.add_argument('--model', default='resnet101', type=str, metavar='MODEL',
                    help='Name of model to train (default: "countception"')
parser.add_argument('--pretrained', action='store_true', default=False,
                    help='Start with pretrained version of specified network (if avail)')
parser.add_argument('--initial-checkpoint', default='', type=str, metavar='PATH',
                    help='Initialize model from this checkpoint (default: none)')
parser.add_argument('--resume', default='', type=str, metavar='PATH',
                    help='Resume full model and optimizer state from checkpoint (default: none)')
parser.add_argument('--no-resume-opt', action='store_true', default=False,
                    help='prevent resume of optimizer state when resuming model')
parser.add_argument('--num-classes', type=int, default=None, metavar='N',
                    help='number of label classes (Model default if None)')
parser.add_argument('--gp', default=None, type=str, metavar='POOL',
                    help='Global pool type, one of (fast, avg, max, avgmax, avgmaxc). Model default if None.')
parser.add_argument('--img-size', type=int, default=None, metavar='N',
                    help='Image patch size (default: None => model default)')
parser.add_argument('--input-size', default=None, nargs=3, type=int,
                    metavar='N N N',
                    help='Input all image dimensions (d h w, e.g. --input-size 3 224 224), uses model default if empty')
parser.add_argument('--crop-pct', default=None, type=float,
                    metavar='N', help='Input image center crop percent (for validation only)')
parser.add_argument('--mean', type=float, nargs='+', default=None, metavar='MEAN',
                    help='Override mean pixel value of dataset')
parser.add_argument('--std', type=float, nargs='+', default=None, metavar='STD',
                    help='Override std deviation of of dataset')
parser.add_argument('--interpolation', default='', type=str, metavar='NAME',
                    help='Image resize interpolation type (overrides model)')
parser.add_argument('-b', '--batch-size', type=int, default=32, metavar='N',
                    help='input batch size for training (default: 32)')
parser.add_argument('-vb', '--validation-batch-size-multiplier', type=int, default=1, metavar='N',
                    help='ratio of validation batch size to training batch size (default: 01)')

# Optimizer parameters
parser.add_argument('--opt', default='sgd', type=str, metavar='OPTIMIZER',
                    help='Optimizer (default: "sgd"')
parser.add_argument('--opt-eps', default=None, type=float, metavar='EPSILON',
                    help='Optimizer Epsilon (default: None, use opt default)')
parser.add_argument('--opt-betas', default=None, type=float, nargs='+', metavar='BETA',
                    help='Optimizer Betas (default: None, use opt default)')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                    help='Optimizer momentum (default: 0.9)')
parser.add_argument('--weight-decay', type=float, default=0.0001,
                    help='weight decay (default: 0.0001)')
parser.add_argument('--clip-grad', type=float, default=None, metavar='NORM',
                    help='Clip gradient norm (default: None, no clipping)')
parser.add_argument('--clip-mode', type=str, default='norm',
                    help='Gradient clipping mode. One of ("norm", "value", "agc")')

# Learning rate schedule parameters
parser.add_argument('--sched', default='step', type=str, metavar='SCHEDULER',
                    help='LR scheduler (default: "step"')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--lr-noise', type=float, nargs='+', default=None, metavar='pct, pct',
                    help='learning rate noise on/off epoch percentages')
parser.add_argument('--lr-noise-pct', type=float, default=0.67, metavar='PERCENT',
                    help='learning rate noise limit percent (default: 0.67)')
parser.add_argument('--lr-noise-std', type=float, default=1.0, metavar='STDDEV',
                    help='learning rate noise std-dev (default: 01.0)')
parser.add_argument('--lr-cycle-mul', type=float, default=1.0, metavar='MULT',
                    help='learning rate cycle len multiplier (default: 01.0)')
parser.add_argument('--lr-cycle-limit', type=int, default=1, metavar='N',
                    help='learning rate cycle limit')
parser.add_argument('--warmup-lr', type=float, default=0.0001, metavar='LR',
                    help='warmup learning rate (default: 0.0001)')
parser.add_argument('--min-lr', type=float, default=1e-5, metavar='LR',
                    help='lower lr bound for cyclic schedulers that hit 0 (1e-5)')
parser.add_argument('--epochs', type=int, default=200, metavar='N',
                    help='number of epochs to train (default: 2)')
parser.add_argument('--epoch-repeats', type=float, default=0., metavar='N',
                    help='epoch repeat multiplier (number of times to repeat dataset epoch per train epoch).')
parser.add_argument('--start-epoch', default=None, type=int, metavar='N',
                    help='manual epoch number (useful on restarts)')
parser.add_argument('--decay-epochs', type=float, default=30, metavar='N',
                    help='epoch interval to decay LR')
parser.add_argument('--warmup-epochs', type=int, default=3, metavar='N',
                    help='epochs to warmup LR, if scheduler supports')
parser.add_argument('--cooldown-epochs', type=int, default=10, metavar='N',
                    help='epochs to cooldown LR at min_lr, after cyclic schedule ends')
parser.add_argument('--patience-epochs', type=int, default=10, metavar='N',
                    help='patience epochs for Plateau LR scheduler (default: 10')
parser.add_argument('--decay-rate', '--dr', type=float, default=0.1, metavar='RATE',
                    help='LR decay rate (default: 0.01)')

# Augmentation & regularization parameters
parser.add_argument('--no-aug', action='store_true', default=False,
                    help='Disable all training augmentation, override other train aug args')
parser.add_argument('--scale', type=float, nargs='+', default=[0.08, 1.0], metavar='PCT',
                    help='Random resize scale (default: 0.08 01.0)')
parser.add_argument('--ratio', type=float, nargs='+', default=[3. / 4., 4. / 3.], metavar='RATIO',
                    help='Random resize aspect ratio (default: 0.75 01.33)')
parser.add_argument('--hflip', type=float, default=0.5,
                    help='Horizontal flip training aug probability')
parser.add_argument('--vflip', type=float, default=0.,
                    help='Vertical flip training aug probability')
parser.add_argument('--color-jitter', type=float, default=0.4, metavar='PCT',
                    help='Color jitter factor (default: 0.4)')
parser.add_argument('--aa', type=str, default=None, metavar='NAME',
                    help='Use AutoAugment policy. "v0" or "original". (default: None)'),
parser.add_argument('--aug-splits', type=int, default=0,
                    help='Number of augmentation splits (default: 0, valid: 0 or >=2)')
parser.add_argument('--jsd', action='store_true', default=False,
                    help='Enable Jensen-Shannon Divergence + CE loss. Use with `--aug-splits`.')
parser.add_argument('--reprob', type=float, default=0., metavar='PCT',
                    help='Random erase prob (default: 0.)')
parser.add_argument('--remode', type=str, default='const',
                    help='Random erase mode (default: "const")')
parser.add_argument('--recount', type=int, default=1,
                    help='Random erase count (default: 01)')
parser.add_argument('--resplit', action='store_true', default=False,
                    help='Do not random erase first (clean) augmentation split')
parser.add_argument('--mixup', type=float, default=0.0,
                    help='mixup alpha, mixup enabled if > 0. (default: 0.)')
parser.add_argument('--cutmix', type=float, default=0.0,
                    help='cutmix alpha, cutmix enabled if > 0. (default: 0.)')
parser.add_argument('--cutmix-minmax', type=float, nargs='+', default=None,
                    help='cutmix min/max ratio, overrides alpha and enables cutmix if set (default: None)')
parser.add_argument('--mixup-prob', type=float, default=1.0,
                    help='Probability of performing mixup or cutmix when either/both is enabled')
parser.add_argument('--mixup-switch-prob', type=float, default=0.5,
                    help='Probability of switching to cutmix when both mixup and cutmix enabled')
parser.add_argument('--mixup-mode', type=str, default='batch',
                    help='How to apply mixup/cutmix params. Per "batch", "pair", or "elem"')
parser.add_argument('--mixup-off-epoch', default=0, type=int, metavar='N',
                    help='Turn off mixup after this epoch, disabled if 0 (default: 0)')
parser.add_argument('--smoothing', type=float, default=0.1,
                    help='Label smoothing (default: 0.01)')
parser.add_argument('--train-interpolation', type=str, default='random',
                    help='Training interpolation (random, bilinear, bicubic default: "random")')
parser.add_argument('--drop', type=float, default=0.0, metavar='PCT',
                    help='Dropout rate (default: 0.)')
parser.add_argument('--drop-connect', type=float, default=None, metavar='PCT',
                    help='Drop connect rate, DEPRECATED, use drop-path (default: None)')
parser.add_argument('--drop-path', type=float, default=None, metavar='PCT',
                    help='Drop path rate (default: None)')
parser.add_argument('--drop-block', type=float, default=None, metavar='PCT',
                    help='Drop block rate (default: None)')

# Batch norm parameters (only works with gen_efficientnet based models currently)
parser.add_argument('--bn-tf', action='store_true', default=False,
                    help='Use Tensorflow BatchNorm defaults for models that support it (default: False)')
parser.add_argument('--bn-momentum', type=float, default=None,
                    help='BatchNorm momentum override (if not None)')
parser.add_argument('--bn-eps', type=float, default=None,
                    help='BatchNorm epsilon override (if not None)')
parser.add_argument('--sync-bn', action='store_true',
                    help='Enable NVIDIA Apex or Torch synchronized BatchNorm.')
parser.add_argument('--dist-bn', type=str, default='',
                    help='Distribute BatchNorm stats between nodes after each epoch ("broadcast", "reduce", or "")')
parser.add_argument('--split-bn', action='store_true',
                    help='Enable separate BN layers per augmentation split.')

# Misc
parser.add_argument('--seed', type=int, default=42, metavar='S',
                    help='random seed (default: 42)')
parser.add_argument('--log-interval', type=int, default=50, metavar='N',
                    help='how many batches to wait before logging training status')
parser.add_argument('--recovery-interval', type=int, default=0, metavar='N',
                    help='how many batches to wait before writing recovery checkpoint')
parser.add_argument('--checkpoint-hist', type=int, default=10, metavar='N',
                    help='number of checkpoints to keep (default: 10)')
parser.add_argument('-j', '--workers', type=int, default=4, metavar='N',
                    help='how many training processes to use (default: 01)')
parser.add_argument('--save-images', action='store_true', default=False,
                    help='save images of input bathes every log interval for debugging')
parser.add_argument('--amp', action='store_true', default=False,
                    help='use NVIDIA Apex AMP or Native AMP for mixed precision training')
parser.add_argument('--apex-amp', action='store_true', default=False,
                    help='Use NVIDIA Apex AMP mixed precision')
parser.add_argument('--native-amp', action='store_true', default=False,
                    help='Use Native Torch AMP mixed precision')
parser.add_argument('--channels-last', action='store_true', default=False,
                    help='Use channels_last memory layout')
parser.add_argument('--pin-mem', action='store_true', default=False,
                    help='Pin CPU memory in DataLoader for more efficient (sometimes) transfer to GPU.')
parser.add_argument('--no-prefetcher', action='store_true', default=False,
                    help='disable fast prefetcher')
parser.add_argument('--experiment', default='', type=str, metavar='NAME',
                    help='name of train experiment, name of sub-folder for output')
parser.add_argument('--eval-metric', default='top1', type=str, metavar='EVAL_METRIC',
                    help='Best metric (default: "top1"')
parser.add_argument('--tta', type=int, default=0, metavar='N',
                    help='Test/inference time augmentation (oversampling) factor. 0=None (default: 0)')


def _parse_args():
    # Do we have a config file to parse?
    args_config, remaining = config_parser.parse_known_args()
    if args_config.config:
        with open(args_config.config, 'r') as f:
            cfg = yaml.safe_load(f)
            parser.set_defaults(**cfg)

    # The main arg parser parses the rest of the args, the usual
    # defaults will have been overridden if config file specified.
    args = parser.parse_args(remaining)

    # Cache the args as a text string to save them in the output dir later
    args_text = yaml.safe_dump(args.__dict__, default_flow_style=False)
    return args, args_text


def train(train_csv, eval_csv, output_dir, args, class_to_idx):
    if has_wandb:
        wandb.init(project=args.experiment, config=args)
    else:
        _logger.warning("You've requested to log metrics to wandb but package not found. "
                        "Metrics not being logged to wandb, try `pip install wandb`")

    args.prefetcher = not args.no_prefetcher
    if 'WORLD_SIZE' in os.environ:
        args.distributed = int(os.environ['WORLD_SIZE']) > 1
    args.device = 'cuda:0'
    args.world_size = 1

    _logger.info('Training with a single process on 01 GPUs.')

    # resolve AMP arguments based on PyTorch / Apex availability
    use_amp = None
    if args.amp:
        # `--amp` chooses native amp before apex (APEX ver not actively maintained)
        if has_native_amp:
            args.native_amp = True
        elif has_apex:
            args.apex_amp = True
    if args.apex_amp and has_apex:
        use_amp = 'apex'
    elif args.native_amp and has_native_amp:
        use_amp = 'native'
    elif args.apex_amp or args.native_amp:
        _logger.warning("Neither APEX or native Torch AMP is available, using float32. "
                        "Install NVIDA apex or upgrade to PyTorch 01.6")

    random_seed(args.seed, 0)

    model = create_model(
        args.model,
        pretrained=args.pretrained,
        num_classes=args.num_classes,
        drop_rate=args.drop,
        drop_connect_rate=args.drop_connect,  # DEPRECATED, use drop_path
        drop_path_rate=args.drop_path,
        drop_block_rate=args.drop_block,
        global_pool=args.gp,
        bn_tf=args.bn_tf,
        bn_momentum=args.bn_momentum,
        bn_eps=args.bn_eps,
        checkpoint_path=args.initial_checkpoint)

    _logger.info(
        f'Model {safe_model_name(args.model)} created, param count:{sum([m.numel() for m in model.parameters()])}')

    data_config = resolve_data_config(vars(args), model=model, verbose=True)

    # setup augmentation batch splits for contrastive loss or split bn
    num_aug_splits = 0
    if args.aug_splits > 0:
        assert args.aug_splits > 1, 'A split of 01 makes no sense'
        num_aug_splits = args.aug_splits

    # enable split bn (separate bn stats per batch-portion)
    if args.split_bn:
        assert num_aug_splits > 1 or args.resplit
        model = convert_splitbn_model(model, max(num_aug_splits, 2))

    # move model to GPU, enable channels last layout if set
    model.cuda()

    optimizer = create_optimizer_v2(model, **optimizer_kwargs(cfg=args))

    # setup automatic mixed-precision (AMP) loss scaling and op casting
    amp_autocast = suppress  # do nothing
    loss_scaler = None
    if use_amp == 'apex':
        model, optimizer = amp.initialize(model, optimizer, opt_level='O1')
        loss_scaler = ApexScaler()
        _logger.info('Using NVIDIA APEX AMP. Training in mixed precision.')
    elif use_amp == 'native':
        amp_autocast = torch.cuda.amp.autocast
        loss_scaler = NativeScaler()
        _logger.info('Using native Torch AMP. Training in mixed precision.')
    else:
        _logger.info('AMP not enabled. Training in float32.')

    # optionally resume from a checkpoint
    resume_epoch = None
    if args.resume:
        resume_epoch = resume_checkpoint(
            model, args.resume,
            optimizer=None if args.no_resume_opt else optimizer,
            loss_scaler=None if args.no_resume_opt else loss_scaler,
            log_info=args.local_rank == 0)


    # setup learning rate schedule and starting epoch
    lr_scheduler, num_epochs = create_scheduler(args, optimizer)
    start_epoch = 0
    if args.start_epoch is not None:
        # a specified start_epoch will always override the resume epoch
        start_epoch = args.start_epoch
    elif resume_epoch is not None:
        start_epoch = resume_epoch
    if lr_scheduler is not None and start_epoch > 0:
        lr_scheduler.step(start_epoch)

    _logger.info('Scheduled epochs: {}'.format(num_epochs))

    # read_wav = check_read_wav_from_model_name(args.model)
    first_part_model_name = model_name.split('_')[0]
    second_part_model_name = model_name.split('_')[1]

    read_vgg_features = False

    if first_part_model_name == "speaker" and second_part_model_name == "vgg":
        read_vgg_features = True
        dataset_train = SpeakerVGGDataset(str(train_csv),
                                          parser=ParserCSV(train_csv, class_to_idx=class_to_idx,
                                                           read_vgg_features=True))
        dataset_eval = SpeakerVGGDataset(str(eval_csv),
                                         parser=ParserCSV(eval_csv, class_to_idx=class_to_idx, read_vgg_features=True))
    else:
        dataset_train = ImageDataset(str(train_csv), parser=ParserCSV(train_csv, class_to_idx=class_to_idx))
        dataset_eval = ImageDataset(str(eval_csv), parser=ParserCSV(eval_csv, class_to_idx=class_to_idx))

    # setup mixup / cutmix
    collate_fn = None
    mixup_fn = None
    mixup_active = args.mixup > 0 or args.cutmix > 0. or args.cutmix_minmax is not None


    if mixup_active:
        mixup_args = dict(
            mixup_alpha=args.mixup, cutmix_alpha=args.cutmix, cutmix_minmax=args.cutmix_minmax,
            prob=args.mixup_prob, switch_prob=args.mixup_switch_prob, mode=args.mixup_mode,
            label_smoothing=args.smoothing, num_classes=args.num_classes)
        if args.prefetcher:
            assert not num_aug_splits  # collate conflict (need to support deinterleaving in collate mixup)
            #collate_fn = FastCollateMixup(**mixup_args)
            collate_fn = CustomFastCollateMixup(**mixup_args)
        else:
            mixup_fn = Mixup(**mixup_args)


    # wrap dataset in AugMix helper
    if num_aug_splits > 1:
        dataset_train = AugMixDataset(dataset_train, num_splits=num_aug_splits)

    # create data loaders w/ augmentation pipeline
    train_interpolation = args.train_interpolation
    if args.no_aug or not train_interpolation:
        train_interpolation = data_config['interpolation']

    if read_vgg_features:
        loader_train = create_custom_loader(
            dataset_train,
            input_size=data_config['input_size'],
            batch_size=args.batch_size,
            is_training=True,
            use_prefetcher=args.prefetcher,
            no_aug=args.no_aug,
            re_prob=args.reprob,
            re_mode=args.remode,
            re_count=args.recount,
            re_split=args.resplit,
            scale=args.scale,
            ratio=args.ratio,
            hflip=args.hflip,
            vflip=args.vflip,
            color_jitter=args.color_jitter,
            auto_augment=args.aa,
            num_aug_splits=num_aug_splits,
            interpolation=train_interpolation,
            mean=data_config['mean'],
            std=data_config['std'],
            num_workers=args.workers,
            collate_fn=collate_fn,
            pin_memory=args.pin_mem,
        )

        loader_eval = create_custom_loader(
            dataset_eval,
            input_size=data_config['input_size'],
            batch_size=args.validation_batch_size_multiplier * args.batch_size,
            is_training=False,
            use_prefetcher=args.prefetcher,
            interpolation=data_config['interpolation'],
            mean=data_config['mean'],
            std=data_config['std'],
            num_workers=args.workers,
            crop_pct=data_config['crop_pct'],
            pin_memory=args.pin_mem,
        )

    else:
        loader_train = create_loader(
            dataset_train,
            input_size=data_config['input_size'],
            batch_size=args.batch_size,
            is_training=True,
            use_prefetcher=args.prefetcher,
            no_aug=args.no_aug,
            re_prob=args.reprob,
            re_mode=args.remode,
            re_count=args.recount,
            re_split=args.resplit,
            scale=args.scale,
            ratio=args.ratio,
            hflip=args.hflip,
            vflip=args.vflip,
            color_jitter=args.color_jitter,
            auto_augment=args.aa,
            num_aug_splits=num_aug_splits,
            interpolation=train_interpolation,
            mean=data_config['mean'],
            std=data_config['std'],
            num_workers=args.workers,
            collate_fn=collate_fn,
            pin_memory=args.pin_mem,
        )

        loader_eval = create_loader(
            dataset_eval,
            input_size=data_config['input_size'],
            batch_size=args.validation_batch_size_multiplier * args.batch_size,
            is_training=False,
            use_prefetcher=args.prefetcher,
            interpolation=data_config['interpolation'],
            mean=data_config['mean'],
            std=data_config['std'],
            num_workers=args.workers,
            crop_pct=data_config['crop_pct'],
            pin_memory=args.pin_mem,
        )

    # setup loss function
    if args.jsd:
        assert num_aug_splits > 1  # JSD only valid with aug splits set
        train_loss_fn = JsdCrossEntropy(num_splits=num_aug_splits, smoothing=args.smoothing).cuda()
    elif mixup_active:
        # smoothing is handled with mixup target transform
        train_loss_fn = SoftTargetCrossEntropy().cuda()

    elif args.smoothing:
        train_loss_fn = LabelSmoothingCrossEntropy(smoothing=args.smoothing).cuda()
    else:
        train_loss_fn = nn.CrossEntropyLoss().cuda()
    validate_loss_fn = nn.CrossEntropyLoss().cuda()

    # setup checkpoint saver and eval metric tracking
    eval_metric = args.eval_metric
    best_metric = None
    best_epoch = None
    decreasing = True if eval_metric == 'loss' else False

    saver = CheckpointSaver(
        model=model, optimizer=optimizer, args=args, amp_scaler=loss_scaler,
        checkpoint_dir=output_dir, recovery_dir=output_dir, decreasing=decreasing, max_history=args.checkpoint_hist)
    with open(os.path.join(output_dir, 'args.yaml'), 'w') as f:
        f.write(args_text)
    """

    saver = Saver(
        model=model, optimizer=optimizer, args=args, amp_scaler=loss_scaler,
        checkpoint_dir=output_dir, recovery_dir=output_dir, decreasing=decreasing, max_history=args.checkpoint_hist)
    with open(os.path.join(output_dir, 'args.yaml'), 'w') as f:
        f.write(args_text)
    """

    train_loss_history, val_loss_history, train_acc_history, val_acc_history = [], [], [], []
    if args.resume:
        train_loss_history, val_loss_history, val_acc_history = read_history_from_csv(os.path.dirname(args.resume))

    try:
        for epoch in range(start_epoch, num_epochs):

            train_metrics = train_one_epoch(
                epoch, model, loader_train, optimizer, train_loss_fn, args,
                lr_scheduler=lr_scheduler, saver=saver, output_dir=output_dir,
                amp_autocast=amp_autocast, loss_scaler=loss_scaler, mixup_fn=mixup_fn)

            eval_metrics = validate(model, loader_eval, validate_loss_fn, args, amp_autocast=amp_autocast)

            if args.save_graphs:
                train_loss_history.append(train_metrics["loss"])
                val_loss_history.append(eval_metrics["loss"])
                val_acc_history.append(eval_metrics["top1"])
                update_graph(train_loss_history, val_loss_history, val_acc_history, os.path.join(output_dir, "graphs"))

            if lr_scheduler is not None:
                # step LR for next epoch
                lr_scheduler.step(epoch + 1, eval_metrics[eval_metric])

            if output_dir is not None:
                update_summary(
                    epoch, train_metrics, eval_metrics, os.path.join(output_dir, 'summary.csv'),
                    write_header=best_metric is None)

            if saver is not None:
                # save proper checkpoint with eval metric
                save_metric = eval_metrics[eval_metric]
                best_metric, best_epoch = saver.save_checkpoint(epoch, metric=save_metric)

    except KeyboardInterrupt:
        pass
    if best_metric is not None:
        _logger.info('*** Best metric: {0} (epoch {1})'.format(best_metric, best_epoch))


def train_one_epoch(
        epoch, model, loader, optimizer, loss_fn, args,
        lr_scheduler=None, saver=None, output_dir=None, amp_autocast=suppress,
        loss_scaler=None, mixup_fn=None):
    if args.mixup_off_epoch and epoch >= args.mixup_off_epoch:
        if args.prefetcher and loader.mixup_enabled:
            loader.mixup_enabled = False
        elif mixup_fn is not None:
            mixup_fn.mixup_enabled = False

    second_order = hasattr(optimizer, 'is_second_order') and optimizer.is_second_order
    batch_time_m = AverageMeter()
    data_time_m = AverageMeter()
    losses_m = AverageMeter()

    model.train()

    end = time.time()
    last_idx = len(loader) - 1
    num_updates = epoch * len(loader)
    for batch_idx, (input, target) in enumerate(loader):

        last_batch = batch_idx == last_idx
        data_time_m.update(time.time() - end)

        with amp_autocast():
            output = model(input)
            loss = loss_fn(output, target)


        if isinstance(input, tuple):
            input_size = input[0].size(0)
        else:
            input_size = input.size(0)

        losses_m.update(loss.item(), input_size)


        optimizer.zero_grad()
        if loss_scaler is not None:
            loss_scaler(
                loss, optimizer,
                clip_grad=args.clip_grad, clip_mode=args.clip_mode,
                parameters=model_parameters(model, exclude_head='agc' in args.clip_mode),
                create_graph=second_order)
        else:
            loss.backward(create_graph=second_order)
            if args.clip_grad is not None:
                dispatch_clip_grad(
                    model_parameters(model, exclude_head='agc' in args.clip_mode),
                    value=args.clip_grad, mode=args.clip_mode)
            optimizer.step()

        torch.cuda.synchronize()
        num_updates += 1
        batch_time_m.update(time.time() - end)
        if last_batch or batch_idx % args.log_interval == 0:
            lrl = [param_group['lr'] for param_group in optimizer.param_groups]
            lr = sum(lrl) / len(lrl)

            _logger.info(
                'Train: {} [{:>4d}/{} ({:>3.0f}%)]  '
                'Loss: {loss.val:>9.6f} ({loss.avg:>6.4f})  '
                'Time: {batch_time.val:.3f}s, {rate:>7.2f}/s  '
                '({batch_time.avg:.3f}s, {rate_avg:>7.2f}/s)  '
                'LR: {lr:.3e}  '
                'Data: {data_time.val:.3f} ({data_time.avg:.3f})'.format(
                    epoch,
                    batch_idx, len(loader),
                    100. * batch_idx / last_idx,
                    loss=losses_m,
                    batch_time=batch_time_m,
                    rate=input_size * args.world_size / batch_time_m.val,
                    rate_avg=input_size * args.world_size / batch_time_m.avg,
                    lr=lr,
                    data_time=data_time_m))

            if args.save_images and output_dir:
                torchvision.utils.save_image(
                    input,
                    os.path.join(output_dir, 'train-batch-%d.jpg' % batch_idx),
                    padding=0,
                    normalize=True)

        if saver is not None and args.recovery_interval and (
                last_batch or (batch_idx + 1) % args.recovery_interval == 0):
            saver.save_recovery(epoch, batch_idx=batch_idx)

        if lr_scheduler is not None:
            lr_scheduler.step_update(num_updates=num_updates, metric=losses_m.avg)
        end = time.time()
        # end for

    if hasattr(optimizer, 'sync_lookahead'):
        optimizer.sync_lookahead()

    return OrderedDict([('loss', losses_m.avg)])


def validate(model, loader, loss_fn, args, amp_autocast=suppress, log_suffix=''):
    batch_time_m = AverageMeter()
    losses_m = AverageMeter()
    top1_m = AverageMeter()
    top5_m = AverageMeter()

    model.eval()

    end = time.time()
    last_idx = len(loader) - 1
    with torch.no_grad():
        for batch_idx, (input, target) in enumerate(loader):
            last_batch = batch_idx == last_idx
            if not args.prefetcher:
                input = input.cuda()
                target = target.cuda()
            if args.channels_last:
                input = input.contiguous(memory_format=torch.channels_last)

            with amp_autocast():
                output = model(input)
            if isinstance(output, (tuple, list)):
                output = output[0]

            # augmentation reduction
            reduce_factor = args.tta
            if reduce_factor > 1:
                output = output.unfold(0, reduce_factor, reduce_factor).mean(dim=2)
                target = target[0:target.size(0):reduce_factor]


            loss = loss_fn(output, target)
            acc1, acc5 = accuracy(output, target, topk=(1, 5))

            reduced_loss = loss.data

            torch.cuda.synchronize()
            if isinstance(input, tuple):
                input_size = input[0].size(0)
            else:
                input_size = input.size(0)
            losses_m.update(reduced_loss.item(), input_size)
            top1_m.update(acc1.item(), output.size(0))
            top5_m.update(acc5.item(), output.size(0))

            batch_time_m.update(time.time() - end)
            end = time.time()
            if (last_batch or batch_idx % args.log_interval == 0):
                log_name = 'Test' + log_suffix
                _logger.info(
                    '{0}: [{1:>4d}/{2}]  '
                    'Time: {batch_time.val:.3f} ({batch_time.avg:.3f})  '
                    'Loss: {loss.val:>7.4f} ({loss.avg:>6.4f})  '
                    'Acc@01: {top1.val:>7.4f} ({top1.avg:>7.4f})  '
                    'Acc@5: {top5.val:>7.4f} ({top5.avg:>7.4f})'.format(
                        log_name, batch_idx, last_idx, batch_time=batch_time_m,
                        loss=losses_m, top1=top1_m, top5=top5_m))

    metrics = OrderedDict([('loss', losses_m.avg), ('top1', top1_m.avg), ('top5', top5_m.avg)])

    return metrics


if __name__ == '__main__':
    setup_default_logging()
    args, args_text = _parse_args()
    model_name = ""
    if args.pretrained:
        model_name = '-'.join(
            [safe_model_name(args.model), 'pretrained'])
    else:
        model_name = '-'.join(
            [safe_model_name(args.model), 'scratch'])

    dataset = get_dataset_name_from_path(args.data_train_dir)
    num_classes_experiment = get_classes_name_from_path(args.data_train_dir)
    aug_type = get_aug_type_from_path(args.data_train_dir)
    exp_type = get_exp_type_from_path(args.data_train_dir)

    assert exp_type == "within-corpus" or exp_type == "IEMOCAP_train" or exp_type == "cross-corpus"

    root_output_dir = os.path.join(args.root_output_dir, exp_type, model_name, dataset,
                                   num_classes_experiment, aug_type)
    class_to_idx = get_class_to_idx(dataset, num_classes_experiment)
    folders = [os.path.basename(f.path) for f in os.scandir(args.data_train_dir) if f.is_dir()]

    if exp_type == "within-corpus":
        for idx, subject in enumerate(folders):
            print("TRAIN MODELS FOR TEST FOLDER " + str(idx + 1) + "/" + str(len(folders)))
            output_subject_dir = os.path.join(root_output_dir, subject)
            subject_dir = os.path.join(args.data_train_dir, subject, "train", "datasets")
            subjects_val = [os.path.basename(f.path) for f in os.scandir(subject_dir) if f.is_dir()]
            for idx_val, subject_val in enumerate(subjects_val):
                print("----TRAIN MODELS FOR TEST FOLDER " + str(idx + 1) + "/" + str(len(folders)) + ": VALIDATION " + str(
                    idx_val + 1) + "/" + str(len(subjects_val)))
                current_train_csv = os.path.join(subject_dir, subject_val, "train", "dataset.csv")
                current_eval_csv = os.path.join(subject_dir, subject_val, "validation", "dataset.csv")
                output_sub_subject_dir = os.path.join(output_subject_dir, subject_val)
                if os.path.exists(output_sub_subject_dir):
                    continue
                else:
                    os.makedirs(output_sub_subject_dir)
                    train(current_train_csv, current_eval_csv, output_sub_subject_dir, args, class_to_idx)

    elif exp_type == "IEMOCAP_train":
        for idx, subject in enumerate(folders):
            print("TRAIN MODELS FOR TEST FOLDER " + str(idx + 1) + "/" + str(len(folders)))
            output_subject_dir = os.path.join(root_output_dir, subject)
            subject_train_csv = os.path.join(args.data_train_dir, subject, "train", "dataset.csv")
            subject_val_csv = os.path.join(args.data_train_dir, subject, "validation", "dataset.csv")
            if os.path.exists(output_subject_dir):
                continue
            else:
                os.makedirs(output_subject_dir)
                train(subject_train_csv, subject_val_csv, output_subject_dir, args, class_to_idx)

    elif exp_type == "cross-corpus":
        for idx, corpus in enumerate(folders):
            print("TRAIN MODELS FOR TEST FOLDER " + str(idx + 1) + "/" + str(len(folders)))
            output_corpus_dir = os.path.join(root_output_dir, corpus)
            corpus_train_csv = os.path.join(args.data_train_dir, corpus, "train", "dataset.csv")
            corpus_eval_csv = os.path.join(args.data_train_dir, corpus, "validation", "dataset.csv")

            if os.path.exists(output_corpus_dir):
                continue
            else:
                os.makedirs(output_corpus_dir)
                train(corpus_train_csv, corpus_eval_csv, output_corpus_dir, args, class_to_idx)

    else:
        exit(0)

Overwriting train.py


# EVAL WITHIN CORPUS

In [ ]:
%%writefile eval_within_corpus.py

import os
import yaml
import argparse

from timm.data import create_loader, resolve_data_config, ImageDataset
from timm.models import create_model

import torch

from custom_loader import create_custom_loader
from parser_csv import ParserCSV
from speaker_vgg_dataset import SpeakerVGGDataset
from utils import save_performance, save_confusion_matrix, get_class_labels
import numpy as np
from utils import *
from src import *
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

import logging

# Model to test arguments

config_parser = parser = argparse.ArgumentParser(description='Training Config', add_help=False)
"""
parser.add_argument('-c', '--config', default='', type=str, metavar='FILE',
                    help='YAML config file specifying default arguments')
"""
parser = argparse.ArgumentParser(description='PyTorch Timm Tests')
# Dataset / Model parameters
parser.add_argument('--data_eval_dir', metavar='DIR',
                    help='path to validation')
parser.add_argument('--speaker_embedder_path', metavar='DIR',
                    help='path to speaker embedder for speaker_cct')
parser.add_argument('--gender_embedder_path', metavar='DIR',
                    help='path to gender embedder for speaker_cct')
parser.add_argument('--corpus_embedder_path', metavar='DIR',
                    help='path to corpus embedder for speaker_cct')
parser.add_argument('--model', metavar='NAME', type=str,
                    help='name model timm')
parser.add_argument('--dataset', '-d', metavar='NAME', default='',
                    help='dataset type (default: ImageFolder/ImageTar if empty)')
parser.add_argument('--num-classes', type=int, default=None, metavar='N',
                    help='number of label classes (Model default if None)')
parser.add_argument('--gp', default=None, type=str, metavar='POOL',
                    help='Global pool type, one of (fast, avg, max, avgmax, avgmaxc). Model default if None.')
parser.add_argument('--img-size', type=int, default=None, metavar='N',
                    help='Image patch size (default: None => model default)')
parser.add_argument('--input-size', default=None, nargs=3, type=int,
                    metavar='N N N',
                    help='Input all image dimensions (d h w, e.g. --input-size 3 224 224), uses model default if empty')
parser.add_argument('--crop-pct', default=None, type=float,
                    metavar='N', help='Input image center crop percent (for validation only)')
parser.add_argument('--mean', type=float, nargs='+', default=None, metavar='MEAN',
                    help='Override mean pixel value of dataset')
parser.add_argument('--std', type=float, nargs='+', default=None, metavar='STD',
                    help='Override std deviation of of dataset')
# Augmentation & regularization parameters
parser.add_argument('--drop', type=float, default=0.0, metavar='PCT',
                    help='Dropout rate (default: 0.)')
parser.add_argument('--drop-connect', type=float, default=None, metavar='PCT',
                    help='Drop connect rate, DEPRECATED, use drop-path (default: None)')
parser.add_argument('--drop-path', type=float, default=None, metavar='PCT',
                    help='Drop path rate (default: None)')
parser.add_argument('--drop-block', type=float, default=None, metavar='PCT',
                    help='Drop block rate (default: None)')
# Batch norm parameters (only works with gen_efficientnet based models currently)
parser.add_argument('--bn-tf', action='store_true', default=False,
                    help='Use Tensorflow BatchNorm defaults for models that support it (default: False)')
parser.add_argument('--bn-momentum', type=float, default=None,
                    help='BatchNorm momentum override (if not None)')
parser.add_argument('--bn-eps', type=float, default=None,
                    help='BatchNorm epsilon override (if not None)')
# Misc
parser.add_argument('--pin-mem', action='store_true', default=False,
                    help='Pin CPU memory in DataLoader for more efficient (sometimes) transfer to GPU.')



# Script arguments

parser.add_argument('model_path', metavar='PATH', type=str,
                    help='model to test relative path')
parser.add_argument('--output_dir', default= "./", metavar='PATH', type=str,
                    help='root outputu dir')
parser.add_argument('--experiment', metavar='NAME', type=str,
                    help='experiment type (research | production | cross-corpus)')
parser.add_argument('--augmentation', metavar='NAME',
                    help='augmentation type (no_aug | time_aug | freq_aug | time_freq_aug)')
parser.add_argument('--pretrained', metavar='NAME', type=str, default='scratch',
                    help='model pretrained (pretrained | scratch)')
parser.add_argument('--data_test_path', metavar='PATH', type=str, default='',
                    help='test data absolute path')
parser.add_argument('--interpolation', metavar='NAME', type=str, default='bicubic',
                    help='resize interpolation')
parser.add_argument('--num_workers', metavar='NAME', type=int, default=4,
                    help='number of workers')
_logger = logging.getLogger('inference')

def _parse_args(config_file):
    # Do we have a config file to parse?
    args_config, remaining = config_parser.parse_known_args()
    args_config.config = config_file

    if args_config.config:
        with open(args_config.config, 'r') as f:
            cfg = yaml.safe_load(f)
            parser.set_defaults(**cfg)

    # The main arg parser parses the rest of the args, the usual
    # defaults will have been overridden if config file specified.
    args = parser.parse_args(remaining)

    # Cache the args as a text string to save them in the output dir later
    args_text = yaml.safe_dump(args.__dict__, default_flow_style=False)
    return args, args_text

def main():
    args = parser.parse_args()
    first_folder = [f.path for f in os.scandir(args.model_path) if f.is_dir()][0]
    config_file = os.path.join([f.path for f in os.scandir(first_folder) if f.is_dir()][0], "args.yaml")
    args, args_text = _parse_args(config_file)
    root_dir = "./"

    pretrained_str = ""
    if args.pretrained:
        pretrained_str = "pretrained"
    else:
        pretrained_str = "scratch"

    model = ('-').join([args.model, pretrained_str])
    dataset = get_dataset_name_from_path(args.data_test_path)
    num_classes_experiment = get_classes_name_from_path(args.data_test_path)
    aug_type = get_aug_type_from_path(args.data_test_path)
    exp_type = get_exp_type_from_path(args.data_test_path)

    assert exp_type == "within-corpus" or exp_type == "IEMOCAP_train" or exp_type == "cross-corpus"

    output_dir = os.path.join(args.output_dir, "test", exp_type, model, dataset,
                              num_classes_experiment, aug_type)
    class_to_idx = get_class_to_idx(dataset, num_classes_experiment)

    class_labels = get_class_labels(args.data_train_dir)

    class_labels_in_order = {}
    for c in class_labels:
        class_labels_in_order[c] = class_to_idx[c]
    class_labels_in_order = {k: v for k, v in sorted(class_labels_in_order.items(), key=lambda item: item[1])}

    folders = [os.path.basename(f.path) for f in os.scandir(args.data_test_path) if f.is_dir()]

    accuracy_tot, micro_precision_tot, macro_precision_tot, micro_recall_tot, macro_recall_tot, \
    micro_f1_tot, macro_f1_tot = 0, 0, 0, 0, 0, 0, 0
    cm_tot = np.zeros((args.num_classes, args.num_classes))

    for idx, folder in enumerate(folders):
        print("PROCESS FOLDER " + str(idx+1) + "/" + str(len(folders)))
        performance_output = os.path.join(output_dir, folder)

        if not os.path.exists(performance_output):
            os.makedirs(performance_output)

        performance_output_file_path = os.path.join(performance_output, "performance.yaml")
        cm_output_file_path = os.path.join(performance_output, "conf_matrix.png")

        eval_csv = os.path.join(args.data_test_path, folder, "test", "dataset.csv")
        eval_models_dirs = [f.path for f in os.scandir(os.path.join(args.model_path, folder)) if f.is_dir()]

        accuracy_folder,  micro_precision_folder, macro_precision_folder, micro_recall_folder, macro_recall_folder,\
        micro_f1_folder, macro_f1_folder = 0, 0, 0, 0, 0, 0, 0
        cm_folder = np.zeros((args.num_classes, args.num_classes))

        for eval_model_dir in eval_models_dirs:

            eval_model_path = os.path.join(eval_model_dir, "model_best.pth.tar")
            if not os.path.exists(eval_csv):
                print("TEST DATASET " + eval_csv + " NOT EXISTS")
                return

            if not os.path.exists(eval_model_path):
                print("MODEL NOT FIND: " + eval_model_path)
                return

            if not os.path.exists(output_dir):
                os.makedirs(output_dir)


            model = create_model(
                args.model,
                num_classes=args.num_classes,
                checkpoint_path=eval_model_path,
                pretrained=args.pretrained,
                drop_rate=args.drop,
                drop_connect_rate=args.drop_connect,
                drop_path_rate=args.drop_path,
                drop_block_rate=args.drop_block,
                global_pool=args.gp,
                bn_tf=args.bn_tf,
                bn_momentum=args.bn_momentum,
                bn_eps=args.bn_eps,
                speaker_embedder_path=args.speaker_embedder_path,
                gender_embedder_path=args.gender_embedder_path,
                corpus_embedder_path=args.corpus_embedder_path)

            config = resolve_data_config(vars(args), model=model)
            model.cuda()

            first_part_model_name = args.model.split('_')[0]
            second_part_model_name = args.model.split('_')[1]
            read_vgg_features = False

            # create the train and eval datasets
            """
            if first_part_model_name == "hybrid":
                read_wav = True
                dataset_train = HybridDataset(str(train_csv),
                                             parser=ParserCSV(train_csv, class_to_idx=class_to_idx, read_wav=True))
                dataset_eval = HybridDataset(str(eval_csv),
                                            parser=ParserCSV(eval_csv, class_to_idx=class_to_idx, read_wav=True))
            """
            if first_part_model_name == "speaker" and second_part_model_name == "vgg":
                read_vgg_features = True
                dataset_test = SpeakerVGGDataset(str(eval_csv), parser=ParserCSV(eval_csv, class_to_idx=class_to_idx,
                                                                                 read_vgg_features=True))
            else:
                dataset_test = ImageDataset(str(eval_csv), parser=ParserCSV(eval_csv, class_to_idx=class_to_idx))
            if read_vgg_features:
                loader = create_custom_loader(
                    dataset_test,
                    input_size=config['input_size'],
                    batch_size=1,
                    use_prefetcher=True,
                    interpolation=config['interpolation'],
                    mean=config['mean'],
                    std=config['std'],
                    num_workers=args.num_workers,
                    crop_pct=config['crop_pct'])
            else:
                loader = create_loader(
                    dataset_test,
                    input_size=config['input_size'],
                    batch_size=1,
                    use_prefetcher=True,
                    interpolation=config['interpolation'],
                    mean=config['mean'],
                    std=config['std'],
                    num_workers=args.num_workers,
                    crop_pct=config['crop_pct'])


            model.eval()

            y_test_true, y_test_predicted = [], []

            with torch.no_grad():
                for batch_idx, (input, target) in enumerate(loader):
                    if isinstance(input, tuple):
                        input = (input[0].cuda(), input[1].cuda())
                    else:
                        input = input.cuda()
                    target = target.cuda()
                    labels = model(input)
                    top1 = labels.topk(1)[1].cpu().numpy()

                    y_test_true.append(target.cpu().numpy()[0])
                    y_test_predicted.append(top1[0][0])


            y_test_true = np.array(y_test_true)
            y_test_predicted = np.array(y_test_predicted)

            # calcolo le statistiche
            accuracy_folder += accuracy_score(y_test_true, y_test_predicted)
            micro_precision_folder += precision_score(y_test_true, y_test_predicted, average="micro")
            macro_precision_folder += precision_score(y_test_true, y_test_predicted, average="macro")
            micro_recall_folder += recall_score(y_test_true, y_test_predicted, average="micro")
            macro_recall_folder += recall_score(y_test_true, y_test_predicted, average="macro")
            micro_f1_folder += f1_score(y_test_true, y_test_predicted, average="micro")
            macro_f1_folder += f1_score(y_test_true, y_test_predicted, average="macro")

            cm_folder += confusion_matrix(y_test_true, y_test_predicted, labels=[class_labels_in_order[c] for c in class_labels_in_order],
                                          normalize='true')
            cm_folder[np.isnan(cm_folder)] = 0

            #report = classification_report(y_test_true, y_test_predicted)

        accuracy = accuracy_folder / len(eval_models_dirs)
        micro_precision = micro_precision_folder / len(eval_models_dirs)
        macro_precision = macro_precision_folder / len(eval_models_dirs)
        micro_recall = micro_recall_folder / len(eval_models_dirs)
        macro_recall = macro_recall_folder / len(eval_models_dirs)
        micro_f1 = micro_f1_folder / len(eval_models_dirs)
        macro_f1 = macro_f1_folder / len(eval_models_dirs)

        cm = cm_folder / len(eval_models_dirs)
        sum_of_rows = cm.sum(axis=1)
        cm = cm / sum_of_rows[:, np.newaxis]
        cm[np.isnan(cm)] = 0

        model_stats = {"accuracy": str(accuracy * 100),
                       "micro_precision": str(micro_precision * 100),
                       "micro_recall": str(micro_recall * 100),
                       "micro_f1": str(micro_f1 * 100),
                       "macro_precision": str(macro_precision * 100),
                       "macro_recall": str(macro_recall * 100),
                       "macro_f1": str(macro_f1 * 100), }

        accuracy_tot += accuracy
        micro_precision_tot += micro_precision
        macro_precision_tot += macro_precision
        micro_recall_tot += micro_recall
        macro_recall_tot += macro_recall
        micro_f1_tot += micro_f1
        macro_f1_tot += macro_f1

        cm_tot += cm

        print(model_stats)

        save_performance(model_stats, performance_output_file_path)
        print("Performance folder " + str(len(folders)) + " saved on: " + performance_output_file_path)

        save_confusion_matrix(cm, class_labels_in_order, cm_output_file_path)
        print("Confusion matrix model saved on: " + cm_output_file_path)


    accuracy = accuracy_tot / len(folders)
    micro_precision = micro_precision_tot / len(folders)
    micro_recall = macro_precision_tot / len(folders)
    micro_f1 = micro_recall_tot / len(folders)
    macro_precision = macro_recall_tot / len(folders)
    macro_recall = micro_f1_tot / len(folders)
    macro_f1 = macro_f1_tot / len(folders)

    cm = cm_tot / len(folders)
    sum_of_rows = cm.sum(axis=1)
    cm = cm / sum_of_rows[:, np.newaxis]


    model_stats = {"accuracy": str(accuracy * 100),
                   "micro_precision": str(micro_precision * 100),
                   "micro_recall": str(micro_recall * 100),
                   "micro_f1": str(micro_f1 * 100),
                   "macro_precision": str(macro_precision * 100),
                   "macro_recall": str(macro_recall * 100),
                   "macro_f1": str(macro_f1 * 100), }

    performance_output_file_path = os.path.join(output_dir, "performance.yaml")
    save_performance(model_stats, performance_output_file_path)
    print("Performance model saved on: " + performance_output_file_path)

    cm_output_file_path = os.path.join(output_dir, "conf_matrix.png")
    save_confusion_matrix(cm, class_labels_in_order, cm_output_file_path)
    print("Confusion matrix model saved on: " + cm_output_file_path)


if __name__ == '__main__':
    main()

# RUN SCRIPTS

In [ ]:
!python train.py /content/drive/MyDrive/datasets/within-corpus_csv/EMODB/all_classes/no_aug/datasets -c /content/drive/MyDrive/Colab\ Notebooks/Models/Compact-Transformers/configs/pretrained/cct_7-3x1_cifar100_300epochs.yml --root_output_dir /content/drive/MyDrive/output/train/ --pretrained --model speaker_vgg_cct_7_3x1_32 --num-classes 6 --input-size 3 32 32 --batch-size 32 --dataset '' --no-aug --workers 2 --checkpoint-hist 1 --save_graphs --epochs 100

In [ ]:
!python train.py /content/drive/MyDrive/datasets/within-corpus_csv/EMODB/all_classes/no_aug/datasets -c /content/drive/MyDrive/Colab\ Notebooks/Models/Compact-Transformers/configs/pretrained/cct_7-3x1_cifar100_300epochs.yml --pretrained --model cct_7_3x1_32 --num-classes 6 --input-size 3 32 32 --batch-size 32 --dataset '' --no-aug --workers 2 --checkpoint-hist 1 --save_graphs --epochs 200

In [ ]:
!python train.py /home/ale-ubuntu/Programming_workspace/Datasets/within-corpus/EMODB/all_classes/no_aug -c ../configs/pretrained/cct_7-3x1_cifar100_300epochs.yml --root_output_dir ../output/train/ --pretrained --model speaker_vgg_cct_7_3x1_32 --num-classes 6 --input-size 3 32 32 --batch-size 32 --dataset '' --workers 8 --checkpoint-hist 1 --save_graphs --epochs 200 --no-aug

In [ ]:
!python eval_within_corpus.py ../output/train/within-corpus/speaker_vgg_cct_7_3x1_32-pretrained/EMODB/all_classes/no_aug/ --data_test_path /home/ale-ubuntu/Programming_workspace/Datasets/within-corpus/EMODB/all_classes/no_aug --output_dir ../